In [ ]:
import os
import numpy as np
import pickle
import pandas as pd
import matplotlib.pyplot as plt

solver = "minion"
pd.set_option("display.precision", 2)

model_classifier = False
type_folder = "/single_problem_custom"
dir_logs = "../ML_logs/rf_all/"+solver+type_folder
files = os.listdir(dir_logs)

ground_truth = "../Dataset/results_on_off_"+solver+".csv"
result_df = pd.read_csv(ground_truth)
result_df.drop_duplicates(["Problem", "Policy"], keep="first", inplace=True)
max_time = result_df['SolverTotalTime'].max() + result_df['SavileRowTotalTime'].max()

In [ ]:
targets = dict()
tot_time_without_tab = 0.0
tot_time_with_tab = 0.0
for prob in result_df["Problem"].unique():
    baseline = result_df.query("Problem=='"+prob+"' and Policy=='baseline'")
    if len(baseline) == 0:
        continue
    baseline_time = baseline['SolverTotalTime'].values[0] + baseline['SavileRowTotalTime'].values[0]
    tab2 = result_df.query("Problem=='"+prob+"' and Policy=='2'")
    if len(tab2) == 0:
        continue
    tab2_time = tab2['SolverTotalTime'].values[0] + tab2['SavileRowTotalTime'].values[0]
    baselineTimeOut = baseline["SolverTimeOut"].fillna(1).values+baseline["SavileRowTimeOut"].fillna(1).values
    tab2TimeOut = tab2["SolverTimeOut"].fillna(1).values+tab2["SavileRowTimeOut"].fillna(1).values
    if baselineTimeOut>0 and tab2TimeOut>0:
        if model_classifier:
            targets[prob] = {'Target': 0, 'Score': 0.0, 'Tab_t': 3600, 'Base_t': 3600}
        else:
            continue  # drop row
    if baselineTimeOut>0 and tab2TimeOut==0:
        clipped = np.clip(tab2_time, 0, 3600)
        targets[prob] = {'Target': 1, 'Score': 3600-clipped, 'Tab_t': clipped, 'Base_t': 3600}
    elif baselineTimeOut==0 and tab2TimeOut>0:
        clipped = np.clip(baseline_time, 0, 3600)
        targets[prob] = {'Target': 0, 'Score': clipped-3600, 'Tab_t': 3600, 'Base_t': clipped}
    elif baselineTimeOut==0 and tab2TimeOut==0 and np.abs(tab2_time-baseline_time)<1:
        if model_classifier:
            label = 1 if tab2_time < baseline_time else 0
            clipped_1 = np.clip(tab2_time, 0, 3600)
            clipped_2 = np.clip(baseline_time, 0, 3600)
            targets[prob] = {'Target': label, 'Score': clipped_2-clipped_1, 'Tab_t': clipped_1, 'Base_t': clipped_2}
        else:
            continue # Skip small differences
    elif baselineTimeOut==0 and tab2TimeOut==0:
        label = 1 if tab2_time < baseline_time else 0
        clipped_1 = np.clip(tab2_time, 0, 3600)
        clipped_2 = np.clip(baseline_time, 0, 3600)
        targets[prob] = {'Target': label, 'Score': clipped_2-clipped_1, 'Tab_t': clipped_1, 'Base_t': clipped_2}
target_df = pd.DataFrame.from_dict(targets, orient ='index')
target_df['Problem'] = target_df.index

In [ ]:
target_df['similar'] = target_df.apply(lambda row: 1 if (row['Tab_t']<=60 and row['Base_t']<=60) or np.abs(row['Tab_t'] - row['Base_t']) < 0.1*min([row['Base_t'], row['Tab_t']]) else 0, axis=1)

In [ ]:
gnn_flag = False
name = dir_logs.split("/")[-1].split("_")[0]
dfs = dict()
if (name == "instance" or name == "problem") and not gnn_flag:
    problems = []
    for file in files:
        split = str(file.split("_")[-1].split(".pkl")[0])
        if split not in dfs.keys():
            dfs[split] = pd.DataFrame()
        filename = os.path.join(dir_logs, file)
        with open(filename, "rb") as pkl_f:
            test_problems, train_problems, all_test, all_train = pickle.load(pkl_f)
        problems.append([test_problems, train_problems])
        dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
        dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_train)])
    for split in dfs.keys():
        dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
elif name == "single" and not gnn_flag:
    for file in files:
        split = str(file.split("_")[-1].split(".pkl")[0])
        if split not in dfs.keys():
            dfs[split] = pd.DataFrame()
        complete_df = pd.DataFrame()
        filename = os.path.join(dir_logs, file)
        with open(filename, "rb") as pkl_f:
            all_test, all_train = pickle.load(pkl_f)
        dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
        dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_train)])
        dfs[split].reset_index().drop(["index"], axis=1)
elif name == "leave" and not gnn_flag:
    complete_df = pd.DataFrame()
    for file in files:
        filename = os.path.join(dir_logs, file)
        with open(filename, "rb") as pkl_f:
            all_test = pickle.load(pkl_f)[0]
        complete_df = pd.concat([complete_df, pd.DataFrame(all_test)])
    df = complete_df.reset_index().drop(["index"], axis=1)
    dfs['0.1'] = df
elif name == "instance" and gnn_flag:
    for file in files:
        split = str(file.split("_")[-1].split(".pkl")[0])
        if split not in dfs.keys():
            dfs[split] = pd.DataFrame()
        filename = os.path.join(dir_logs, file)
        with open(filename, "rb") as pkl_f:
            all_test = pickle.load(pkl_f)[0]
        all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
        all_test["prediction"] = [0 for x in range(len(all_test["Problem"]))]
        dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
    for split in dfs.keys():
        dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
elif name == "leave" and gnn_flag:
    complete_df = pd.DataFrame()
    for file in files:
        filename = os.path.join(dir_logs, file)
        with open(filename, "rb") as pkl_f:
            all_test = pickle.load(pkl_f)[0]
        all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
        if len(all_test["prediction"][0][0])==1:
            all_test["prediction"] = [[x[0] for x in all_test["prediction"][i]] for i in range(len(all_test["prediction"]))]
        complete_df = pd.concat([complete_df, pd.DataFrame(all_test)])
    df = complete_df.reset_index().drop(["index"], axis=1)
    dfs['0.1'] = df
elif name == "problem" and gnn_flag:
    for file in files:
        split = str(file.split("_")[-1].split(".pkl")[0])
        if split not in dfs.keys():
            dfs[split] = pd.DataFrame()
        filename = os.path.join(dir_logs, file)
        with open(filename, "rb") as pkl_f:
            all_test = pickle.load(pkl_f)[0]
        all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
        all_test["prediction"] = [0 for x in range(len(all_test["Problem"]))]
        dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
    for split in dfs.keys():
        dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
elif name == "single" and gnn_flag:
    for file in files:
        split = str(file.split("_")[-1].split(".pkl")[0])
        if split not in dfs.keys():
            dfs[split] = pd.DataFrame()
        complete_df = pd.DataFrame()
        filename = os.path.join(dir_logs, file)
        with open(filename, "rb") as pkl_f:
            all_test = pickle.load(pkl_f)[0]
        all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
        if len(all_test["prediction"][0][0])==1:
            all_test["prediction"] = [[x[0] for x in all_test["prediction"][i]] for i in range(len(all_test["prediction"]))]
        dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
        dfs[split].reset_index().drop(["index"], axis=1)

In [ ]:
if name == "instance" or name == "problem":
    for df_key in dfs.keys():
        complete_df = dfs[df_key]
        df = pd.DataFrame()
        num_cols = [x for x in complete_df.columns if x not in ["Problem", "train_test", "prediction"]]
        for prob in complete_df['Problem'].unique():
            prob_df = complete_df.query("Problem=='"+prob+"'")
            for train_test in prob_df['train_test'].unique():
                a = prob_df.query("train_test=='"+train_test+"'")
                tmp_df = a[num_cols].mean().to_frame().T
                tmp_df["Problem"] = prob
                tmp_df["train_test"] = train_test
                tmp_df["prediction"] = 1 if (a["prediction"]==a["Problem"]).sum()>=len(a["Problem"])/2 else 0
                df = pd.concat([df, tmp_df])
        dfs[df_key] = df.reset_index().drop(["index"], axis=1)
elif name == "single" or name=="leave":
    for df_key in dfs.keys():
        print(len(dfs[df_key]))
        dfs[df_key] = dfs[df_key].groupby(["Problem", "train_test"]).agg({
            "saved_v_best": 'mean',
            "saved_class": 'mean',
            "saved_heur": 'mean',
            "tot_best": 'mean',
            "tot_class": 'mean',
            "tot_heur": 'mean',
            "tot_no_tab": 'mean'
        })
        dfs[df_key] = dfs[df_key].reset_index()
        print(len(dfs[df_key]))
if gnn_flag and "instance" in dfs.keys():
    dfs['0.3'] = dfs["instance"]
elif gnn_flag and "problem" in dfs.keys():
    dfs['0.3'] = dfs["problem"]
for df_key in dfs.keys():
    sums_by_split = dfs[df_key].query("train_test=='test'").drop(["Problem", "train_test"], axis = 1).sum()
    if sums_by_split["saved_heur"]>0:
        sums_by_split["gap"] = 100*(sums_by_split["tot_class"]-sums_by_split["tot_heur"])/(sums_by_split["tot_best"]-sums_by_split["tot_heur"])
    else:
        sums_by_split["gap"] = 100*(sums_by_split["tot_class"]-sums_by_split["tot_no_tab"])/(sums_by_split["tot_best"]-sums_by_split["tot_no_tab"])
    print("Split: ", df_key, "\n", sums_by_split, "\n")

In [ ]:
best_key = "0.1"
def get_metrics(tab_act_num=1):
    target_df["pred"] = -1
    if name in ["leave", "single"]:
        probs_stats = pd.DataFrame()
        for _, row in dfs[best_key].query("train_test == 'test'").iterrows():
            prob_stats_dict = {"fp": [0], "fp_time": [0.0], "fn": [0], "fn_time": [0.0], 
                               "no_diff": [0], "no_diff_time": [0.0], "Problem": row["Problem"], 
                               "tp": [0], "tp_time": [0.0], "tn": [0], "tn_time": [0.0], "tp_heur": [0.0], 
                               "fp_heur": [0.0], "tn_heur": [0.0], "fn_heur": [0.0], "disagreement_time": [0.0], 
                               "agreement_time": [0.0]}
            for label_instance in row["prediction"]:
                instance = label_instance[4:]
                target_df.loc[instance, "pred"] = 1 if label_instance[:3]=="pos" else 0
                stats = target_df.query("Problem == '"+ instance+ "'")
                if len(stats) <1:
                    continue
                fp_flag = stats["pred"].values[0]==1 and stats["Target"].values[0] == 0
                fn_flag = stats["pred"].values[0]==0 and stats["Target"].values[0] == 1
                tp_flag = stats["pred"].values[0]==1 and stats["Target"].values[0] == 1
                tn_flag = stats["pred"].values[0]==0 and stats["Target"].values[0] == 0
                if tab_act_num != -1:
                    tab_activated = result_df.query("Problem == '"+instance+"'")["TabulationActivated"+str(tab_act_num)].dropna()
                else:
                    tab_activated = result_df.query("Problem == '"+instance+"'")["TabulationActivated"].dropna()
                if len(tab_activated) <1:
                    continue
                tab_activated = [1 if x>0 else 0 for x in tab_activated]
                tp_heur = tab_activated[0] == 1 and stats["pred"].values[0]==1
                fp_heur = tab_activated[0] == 0 and stats["pred"].values[0]==1
                tn_heur = tab_activated[0] == 0 and stats["pred"].values[0]==0
                fn_heur = tab_activated[0] == 1 and stats["pred"].values[0]==0
                if fp_flag and stats["similar"].values[0] == 0:
                    prob_stats_dict["fp"][0] += 1
                    prob_stats_dict["fp_time"][0] += np.abs(stats["Score"].values[0])
                elif fn_flag and stats["similar"].values[0] == 0:
                    prob_stats_dict["fn"][0] += 1
                    prob_stats_dict["fn_time"][0] += np.abs(stats["Score"].values[0])
                elif (fp_flag or fn_flag) and stats["similar"].values[0] == 1:
                    prob_stats_dict["no_diff"][0] += 1
                    prob_stats_dict["no_diff_time"][0] += np.abs(stats["Score"].values[0])
                elif tp_flag:
                    prob_stats_dict["tp"][0] += 1
                    prob_stats_dict["tp_time"][0] += np.abs(stats["Score"].values[0])
                elif tn_flag:
                    prob_stats_dict["tn"][0] += 1
                    prob_stats_dict["tn_time"][0] += np.abs(stats["Score"].values[0])
                if fp_heur:
                    prob_stats_dict["fp_heur"][0] += 1
                    prob_stats_dict["disagreement_time"][0] += stats["Tab_t"].values[0]-(stats["Base_t"].values[0]-stats["pred"].values[0]*stats["Score"].values[0])
                elif fn_heur:
                    prob_stats_dict["fn_heur"][0] += 1
                    prob_stats_dict["disagreement_time"][0] += stats["Tab_t"].values[0]-(stats["Base_t"].values[0]-stats["pred"].values[0]*stats["Score"].values[0])
                elif tp_heur:
                    prob_stats_dict["tp_heur"][0] += 1
                    prob_stats_dict["agreement_time"][0] += stats["Tab_t"].values[0]-(stats["Base_t"].values[0]-stats["pred"].values[0]*stats["Score"].values[0])
                elif tn_heur:
                    prob_stats_dict["tn_heur"][0] += 1
                    prob_stats_dict["agreement_time"][0] += stats["Tab_t"].values[0]-(stats["Base_t"].values[0]-stats["pred"].values[0]*stats["Score"].values[0])
            prob_stats_dict["precision"]=prob_stats_dict["tp"][0]/(prob_stats_dict["tp"][0]+prob_stats_dict["fp"][0]+0.000001)
            prob_stats_dict["recall"]=prob_stats_dict["tp"][0]/(prob_stats_dict["tp"][0]+prob_stats_dict["fn"][0]+0.000001)
            prob_stats_dict["f1"]=2*prob_stats_dict["recall"]*prob_stats_dict["precision"]/(prob_stats_dict["recall"]+prob_stats_dict["precision"]+0.000001)
            probs_stats = pd.concat([probs_stats, pd.DataFrame.from_dict(prob_stats_dict)])
        probs_stats = probs_stats.reset_index()
        probs_stats.drop(["Problem"], axis=1).describe()
    elif name in ["problem", "instance"]:
        probs_stats = pd.DataFrame()
        done = []
        for _, row in dfs[best_key].query("train_test == 'test'").iterrows():
            curr_prob = row["Problem"].split("_")[0]
            prob_stats_dict = {"fp": [0], "fp_time": [0.0], "fn": [0], "fn_time": [0.0], 
                               "no_diff": [0], "no_diff_time": [0.0], "tp": [0], "tp_time": [0.0],
                               "tn": [0], "tn_time": [0.0], "tp_heur": [0.0], 
                               "fp_heur": [0.0], "tn_heur": [0.0], "fn_heur": [0.0], "disagreement_time": [0.0], 
                               "agreement_time": [0.0]}
            fp_flag = row["saved_v_best"]==0 and row["saved_heur"]<0 and row["saved_class"] < row["saved_heur"]
            fn_flag = row["saved_v_best"]>0 and row["saved_heur"]>0 and row["saved_class"] <= 0
            tp_flag = row["saved_v_best"]>0 and row["saved_heur"]>0 and row["saved_class"] > 0
            tn_flag = row["saved_v_best"]==0 and row["saved_heur"]<0 and row["saved_class"] <= 0
            if tab_act_num == -1:
                tab_activated = result_df.query("Problem == '"+row["Problem"]+"' and Policy=='2'")["TabulationActivated"].dropna()
            else:
                tab_activated = result_df.query("Problem == '"+row["Problem"]+"' and Policy=='2'")["TabulationActivated"+str(tab_act_num)].dropna()
            if len(tab_activated) <1:
                continue
            tab_activated = [1 if x>0 else 0 for x in tab_activated]
            tp_heur = tab_activated[0] == 1 and (tp_flag or fp_flag)
            fp_heur = tab_activated[0] == 0 and (tp_flag or fp_flag)
            tn_heur = tab_activated[0] == 0 and (tn_flag or fn_flag)
            fn_heur = tab_activated[0] == 1 and (tn_flag or fn_flag)
            if fp_flag:
                prob_stats_dict["fp"][0] += 1
            elif fn_flag:
                prob_stats_dict["fn"][0] += 1
            elif tp_flag:
                prob_stats_dict["tp"][0] += 1
            elif tn_flag:
                prob_stats_dict["tn"][0] += 1
            if fp_heur:
                prob_stats_dict["fp_heur"][0] += 1
                prob_stats_dict["disagreement_time"][0] += row["saved_class"]-row["saved_heur"]
            elif fn_heur:
                prob_stats_dict["fn_heur"][0] += 1
                prob_stats_dict["disagreement_time"][0] += row["saved_class"]-row["saved_heur"]
            elif tp_heur:
                prob_stats_dict["tp_heur"][0] += 1
                prob_stats_dict["agreement_time"][0] += row["saved_class"]-row["saved_heur"]
            elif tn_heur:
                prob_stats_dict["tn_heur"][0] += 1
                prob_stats_dict["agreement_time"][0] += row["saved_class"]-row["saved_heur"]
            prob_stats_dict["precision"]=prob_stats_dict["tp"][0]/(prob_stats_dict["tp"][0]+prob_stats_dict["fp"][0]+0.000001)
            prob_stats_dict["recall"]=prob_stats_dict["tp"][0]/(prob_stats_dict["tp"][0]+prob_stats_dict["fn"][0]+0.000001)
            prob_stats_dict["f1"]=2*prob_stats_dict["recall"]*prob_stats_dict["precision"]/(prob_stats_dict["recall"]+prob_stats_dict["precision"]+0.000001)
            probs_stats = pd.concat([probs_stats, pd.DataFrame.from_dict(prob_stats_dict)])
        probs_stats = probs_stats.reset_index()
    return probs_stats, target_df[target_df["pred"]!=-1].copy()

if "TabulationActivated1" in result_df.columns:
    agreement_dict = dict()
    disagreement_dict = dict()
    for i in [1, 2, 3, 4, 5]:
        probs_stats_tmp, target_df_tmp = get_metrics(tab_act_num=i)
        agreement_dict[i] = probs_stats_tmp["agreement_time"].sum()
        disagreement_dict[i] = probs_stats_tmp["disagreement_time"].sum()
        if i == 1:
            probs_stats = probs_stats_tmp
            if not model_classifier:
                target_df = target_df_tmp
else:
    probs_stats_tmp, target_df_tmp = get_metrics(tab_act_num=-1)
    probs_stats = probs_stats_tmp
    if not model_classifier:
        target_df = target_df_tmp

In [ ]:
if name in ["leave", "single"]:
    print("Total false positives: ", probs_stats["fp"].sum(), ", Cost of fp: ", np.abs(probs_stats["fp_time"]).sum())
    print("Total false negatives: ", probs_stats["fn"].sum(), ", Cost of fn: ", np.abs(probs_stats["fn_time"]).sum())
    print("Total no diff: ", probs_stats["no_diff"].sum(), ", Cost of no_diff: ", np.abs(probs_stats["no_diff_time"]).sum())
    print("Total true positives: ", probs_stats["tp"].sum(), ", Saved with tp: ", np.abs(probs_stats["tp_time"]).sum())
    print("Total true negatives: ", probs_stats["tn"].sum(), ", Time of tn: ", np.abs(probs_stats["tn_time"]).sum())
    print("Mean precision: ", probs_stats["precision"].mean())
    print("Mean recall: ", probs_stats["recall"].mean())
    print("Mean f1: ", probs_stats["f1"].mean())
    print("Agreement heuristics-ML: ", probs_stats["tp_heur"].sum()+probs_stats["tn_heur"].sum())
    print("Disagreement heuristics-ML: ", probs_stats["fn_heur"].sum()+probs_stats["fp_heur"].sum())
    print("Disagreement time saved: ", probs_stats["disagreement_time"].sum())
    print("Agreement time saved: ", probs_stats["agreement_time"].sum())
    probs_stats.query("fp>0 or fn>0 or no_diff>0").drop(["index"], axis=1).reset_index().drop(["index"], axis=1)
else:
    print("Total false positives: ", probs_stats["fp"].sum())
    print("Total false negatives: ", probs_stats["fn"].sum())
    print("Total true positives: ", probs_stats["tp"].sum())
    print("Total true negatives: ", probs_stats["tn"].sum())
    print("Mean precision: ", probs_stats["precision"].mean())
    print("Mean recall: ", probs_stats["recall"].mean())
    print("Mean f1: ", probs_stats["f1"].mean())
    print("Agreement heuristics-ML: ", probs_stats["tp_heur"].sum()+probs_stats["tn_heur"].sum())
    print("Disagreement heuristics-ML: ", probs_stats["fn_heur"].sum()+probs_stats["fp_heur"].sum())
    print("Disagreement time saved: ", probs_stats["disagreement_time"].sum())
    print("Agreement time saved: ", probs_stats["agreement_time"].sum())

In [ ]:
if name == "single" or name=="leave":
    seconds = 60
    if not gnn_flag:
        titles = {"saved_class": "Proportion of time saved for each problem with random forest classifier", 
                 "saved_v_best": "Proportion of time saved for each problem with virtual best classifier"}
    else:
        titles = {"saved_class": "Proportion of time saved for each problem with Graph Neural Network", 
                 "saved_v_best": "Proportion of time saved for each problem with virtual best classifier"}
    saved_thresh = ", when the time saved is more than "+str(seconds)+" seconds."
    total_thresh = ", when the total time is more than "+str(seconds)+" seconds."
    for col in ["saved_class", "saved_v_best"]:
        for ext in [".", "thresh"]:
            if ext == "thresh" and col.split("_")[0] == "saved":
                title = titles[col] + saved_thresh
                thresh = seconds
            elif ext == "thresh" and col.split("_")[0] == "tot":
                title = titles[col] + total_thresh
                thresh = seconds
            elif ext == ".":
                title = titles[col] + "."
                thresh = 0
            a = dfs[best_key].query("train_test == 'test'")[["Problem", col, "tot_heur", "tot_no_tab"]]
            a = a[a[col]>thresh]
            b = a["tot_heur"]  # no_tab"]
            a[col] = b-(a["tot_no_tab"]-a[col])  # saved with respect to the heuristics
            a = a[["Problem", col]]
            plt.figure(figsize=(20, 6))
            plt.bar(np.arange(0, len(a["Problem"]), 1), a[col]/b*100)
            plt.title(title)
            plt.xlabel("Problem")
            plt.ylabel("Proportion of time saved with respect to always using the heuristics.")  # the baseline.")
            plt.xticks(ticks=np.arange(0, len(a["Problem"]), 1), labels=a["Problem"]);
            plt.savefig(col+"_"+ext+"_heur.png", bbox_inches = 'tight')

In [ ]:
if name == "single" or name=="leave":
    seconds = 60
    if not gnn_flag:
        titles = {"saved_class": "Proportion of time saved for each problem with random forest classifier", 
                 "saved_v_best": "Proportion of time saved for each problem with virtual best classifier"}
    else:
        titles = {"saved_class": "Proportion of time saved for each problem with Graph Neural Network", 
                 "saved_v_best": "Proportion of time saved for each problem with virtual best classifier"}
    saved_thresh = ", when the time saved is more than "+str(seconds)+" seconds."
    total_thresh = ", when the total time is more than "+str(seconds)+" seconds."
    for col in ["saved_class", "saved_v_best"]:
        for ext in [".", "thresh"]:
            if ext == "thresh" and col.split("_")[0] == "saved":
                title = titles[col] + saved_thresh
                thresh = seconds
            elif ext == "thresh" and col.split("_")[0] == "tot":
                title = titles[col] + total_thresh
                thresh = seconds
            elif ext == ".":
                title = titles[col] + "."
                thresh = 0
            a = dfs[best_key].query("train_test == 'test'")[["Problem", col, "tot_no_tab"]]
            if thresh == 0:
                a = a[abs(a[col])>1000]
            else:
                a = a[a[col]>thresh]
            b = a["tot_no_tab"]
            a = a[["Problem", col]]
            plt.figure(figsize=(20, 6))
            plt.bar(np.arange(0, len(a["Problem"]), 1), a[col]/b*100)
            plt.title(title)
            plt.xlabel("Problem")
            plt.ylabel("Proportion of time saved with respect to always using the baseline.")
            plt.xticks(ticks=np.arange(0, len(a["Problem"]), 1), labels=a["Problem"]);
            plt.savefig(col+"_"+ext+"_base.png", bbox_inches = 'tight')

In [ ]:
if name == "single" or name=="leave":
    seconds = 60
    if not gnn_flag:
        titles = {"tot_heur": "Total time for each problem when always using the heuristics", 
                 "tot_no_tab": "Total time for each problem without tabulation."}
    else:
        titles = {"tot_heur": "Total time for each problem when always using the heuristics", 
                 "tot_no_tab": "Total time for each problem without tabulation."}
    saved_thresh = ", when the time saved is more than "+str(seconds)+" seconds."
    total_thresh = ", when the total time is more than "+str(seconds)+" seconds."
    for col in ["tot_heur", "tot_no_tab"]:
        for ext in [".", "thresh"]:
            if ext == "thresh" and col.split("_")[0] == "saved":
                title = titles[col] + saved_thresh
                thresh = seconds
            elif ext == "thresh" and col.split("_")[0] == "tot":
                title = titles[col] + total_thresh
                thresh = seconds
            elif ext == ".":
                title = titles[col] + "."
                thresh = 0
            a = dfs[best_key].query("train_test == 'test'")[["Problem", col]]
            a = a[a[col]>thresh]
            plt.figure(figsize=(20, 6))
            plt.bar(np.arange(0, len(a["Problem"]), 1), np.log(a[col]))
            plt.title(title)
            plt.xlabel("Problem")
            plt.ylabel("Time (s), logaritmic scale")
            plt.xticks(ticks=np.arange(0, len(a["Problem"]), 1), labels=a["Problem"]);
            plt.savefig(col+"_"+ext+".png", bbox_inches = 'tight')

In [ ]:
for df_key in [best_key]:  # dfs.keys():
    print("Split: ", df_key, "\n")
    a = dfs[df_key].query("train_test=='test'").drop(["Problem", "train_test"], axis = 1).sum()
    b = a[['saved_v_best', 'saved_class', 'saved_heur']]
    c = a[['tot_best', 'tot_class', 'tot_heur', 'tot_no_tab']]
    b.plot(kind='bar')
    plt.title("Total time saved with solver: " + solver[0].upper()+solver[1:])
    plt.xticks(ticks=np.arange(0,len(b),1), labels=['saved_v_best', 'saved_class', 'saved_heur'])
    plt.savefig("time_saved.png", bbox_inches = 'tight')
    plt.show()
    c.plot(kind='bar')
    plt.title("Total time with solver: " + solver[0].upper()+solver[1:])
    plt.xticks(ticks=np.arange(0,len(c),1), labels=['tot_best', 'tot_class', 'tot_heur', 'tot_no_tab'])
    plt.savefig("time_total.png", bbox_inches = 'tight')
    plt.show()

In [ ]:
if name == "instance":
    from utils import calculateSpeedups_2, _2dp, _gm

    fig = plt.figure(figsize=(16, 16))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)
    df = dfs[best_key].copy()
    list_probs = df.query("train_test=='test'").Problem.apply(lambda x: x.split("_")[0]).unique()
    dict_probs = dict()
    for prob in list_probs:
        dict_probs[prob.lower()] = prob
    list_probs = sorted([x.lower() for x in list_probs])
    list_probs = [dict_probs[x] for x in list_probs]
    for i, prob in enumerate(list_probs[:28]):
        ax = fig.add_subplot(7, 4, i+1)
        tmp_df = df[df.Problem.str.contains(prob)].query("train_test=='test'")
        df_speedups = calculateSpeedups_2(tmp_df, False, "saved_class")
        ax.scatter(tmp_df['tot_heur'], tmp_df['tot_class'], c="black", marker="x");
        ax.set_yscale('symlog')
        ax.set_xscale('symlog')
        ax.plot([0, 4800+5], [0, 4800+5], c="black");
        ax.set_xlim(0, 4805)
        ax.set_ylim(0, 4805)
        ax.set_title(prob[0].upper()+prob[1:]);
        facet_speedups = df_speedups["speedup"].to_numpy()
        gm = _2dp(_gm(facet_speedups))
        _lim=(1,1.5 * 3600)
        ax.text(0.5 * _lim[1], 2 * _lim[0], s=f"s = {gm}", ha='right')
    fig.supylabel("Total time in log scale with random forest classifier (s)");
    fig.supxlabel("Total time in log scale when always using the heuristics (s)");
    plt.savefig(solver+"_multi_scatter_1.png")
    plt.show()

    fig = plt.figure(figsize=(16, 16))
    fig.subplots_adjust(hspace=0.5, wspace=0.5)
    df = dfs[best_key]
    for i, prob in enumerate(list_probs[28:52]):
        ax = fig.add_subplot(6, 4, i+1)
        tmp_df = df[df.Problem.str.contains(prob)].query("train_test=='test'")
        df_speedups = calculateSpeedups_2(tmp_df, False, "saved_class")
        ax.scatter(tmp_df['tot_heur'], tmp_df['tot_class'], c="black", marker="x");
        ax.set_yscale('symlog')
        ax.set_xscale('symlog')
        ax.plot([0, 4800+5], [0, 4800+5], c="black");
        ax.set_xlim(0, 4805)
        ax.set_ylim(0, 4805)
        ax.set_title(prob[0].upper()+prob[1:]);
        facet_speedups = df_speedups["speedup"].to_numpy()
        gm = _2dp(_gm(facet_speedups))
        _lim=(1,1.5 * 3600)
        ax.text(0.5 * _lim[1], 2 * _lim[0], s=f"s = {gm}", ha='right')
    fig.supylabel("Total time in log scale with random forest classifier (s)");
    fig.supxlabel("Total time in log scale when always using the heuristics (s)");
    plt.savefig(solver+"_multi_scatter_2.png")
    plt.show()

In [ ]:
for df_key in [best_key]:  # dfs.keys():
    df = dfs[df_key]
    plt.figure(figsize=(8, 8));
    ax = plt.gca()
    if name == "leave" or name == "single":
        max_time = int(max(df.query("train_test=='test'")['tot_best'].max(), df.query("train_test=='test'")['tot_heur'].max()))
    ax.scatter(df.query("train_test=='test'")['tot_heur'], df.query("train_test=='test'")['tot_best'], c="black", marker="x");
    ax.set_yscale('symlog')
    ax.set_xscale('symlog')
    if name == "leave" or name == "single":
        plt.plot([0, max_time+5], [0, max_time+5], c="black");
    else:
        plt.plot([0, 4800+5], [0, 4800+5], c="black");
    if name != 'leave':
        plt.title("Total time for problems in test set with split="+df_key);
        plt.ylabel("Total time of virtual best classifier (s) with split="+df_key);
        plt.xlabel("Total time when always using the heuristics (s) with split="+df_key);
    else:
        plt.title("Total time for problems in test set.");
        plt.ylabel("Total time of virtual best classifier (s).");
        plt.xlabel("Total time when always using the heuristics (s).");
    plt.tight_layout();
    if name == "leave" or name == "single":
        plt.xlim(left=0, right=max_time);
        plt.ylim(bottom=0, top=max_time);
    else:
        plt.xlim(left=0, right=4800);
        plt.ylim(bottom=0, top=4800);
    plt.savefig("best_heuristics.jpg", bbox_inches = 'tight')
    plt.show();

In [ ]:
for df_key in [best_key]:  # dfs.keys():
    df = dfs[df_key]
    plt.figure(figsize=(8, 8));
    ax = plt.gca()
    if name == "leave" or name == "single":
        max_time = int(max(df.query("train_test=='test'")['tot_heur'].max(), df.query("train_test=='test'")['tot_class'].max()))
    ax.scatter(df.query("train_test=='test'")['tot_heur'], df.query("train_test=='test'")['tot_class'], c="black", marker="x");
    ax.set_yscale('symlog')
    ax.set_xscale('symlog')
    if name == "leave" or name == "single":
        plt.plot([0, max_time+5], [0, max_time+5], c="black");
    else:
        plt.plot([0, 4800+5], [0, 4800+5], c="black");
    if name != 'leave':
        plt.title("Total time for problems in test set with split="+df_key);
        plt.ylabel("Total time of random forest classifier (s) with split="+df_key);
        plt.xlabel("Total time when always using the heuristics (s) with split="+df_key);
    else:
        plt.title("Total time for problems in test set.");
        plt.ylabel("Total time of random forest classifier (s).");
        plt.xlabel("Total time when always using the heuristics (s).");
    plt.tight_layout();
    if name == "leave" or name == "single":
        plt.xlim(left=0, right=max_time);
        plt.ylim(bottom=0, top=max_time);
    else:
        plt.xlim(left=0, right=4800);
        plt.ylim(bottom=0, top=4800);
    plt.savefig("class_heuristics_test.jpg", bbox_inches = 'tight')
    plt.show();

In [ ]:
if not gnn_flag:
    for df_key in [best_key]:  # dfs.keys():
        df = dfs[df_key]
        plt.figure(figsize=(8, 8));
        ax = plt.gca()
        if name == "leave" or name == "single":
            max_time = int(max(df.query("train_test=='train'")['tot_heur'].max(), df.query("train_test=='train'")['tot_class'].max()))
        ax.scatter(df.query("train_test=='train'")['tot_heur'], df.query("train_test=='train'")['tot_class'], c="black", marker="x");
        ax.set_yscale('symlog')
        ax.set_xscale('symlog')
        if name == "leave" or name == "single":
            plt.plot([0, max_time+5], [0, max_time+5], c="black");
        else:
            plt.plot([0, 4800+5], [0, 4800+5], c="black");
        if name != 'leave':
            plt.title("Total time for problems in train set with split="+df_key);
            plt.ylabel("Total time of random forest classifier (s) with split="+df_key);
            plt.xlabel("Total time when always using the heuristics (s) with split="+df_key);
        else:
            plt.title("Total time for problems in train set.");
            plt.ylabel("Total time of random forest classifier (s).");
            plt.xlabel("Total time when always using the heuristics (s).");
        plt.tight_layout();
        if name == "leave" or name == "single":
            plt.xlim(left=0, right=max_time);
            plt.ylim(bottom=0, top=max_time);
        else:
            plt.xlim(left=0, right=4800);
            plt.ylim(bottom=0, top=4800);
        plt.savefig("class_heuristics_train.jpg", bbox_inches = 'tight')
        plt.show();

In [ ]:
tmp_solv = "minion"
type_folder = "/single_problem_custom"

def run_setting(tmp_solv, type_folder):
    df_solvs = dict()
    df_solvs_class = dict()
    for tmp_set in ["rf_all", "dt_subset", "rf_subset", "dt_all"]:
        tmp_dir_logs = "../ML_logs/"+tmp_set+"/"+tmp_solv+type_folder
        tmp_files = os.listdir(tmp_dir_logs)
        tmp_ground_truth = "../Dataset/results_on_off_"+tmp_solv+".csv"
        df_solvs[tmp_set] = pd.read_csv(tmp_ground_truth)
        df_solvs[tmp_set].drop_duplicates(["Problem", "Policy"], keep="first", inplace=True)
        targets = dict()
        tot_time_without_tab = 0.0
        tot_time_with_tab = 0.0
        for prob in df_solvs[tmp_set]["Problem"].unique():
            baseline = df_solvs[tmp_set].query("Problem=='"+prob+"' and Policy=='baseline'")
            if len(baseline) == 0:
                continue
            baseline_time = baseline['SolverTotalTime'].values[0] + baseline['SavileRowTotalTime'].values[0]
            tab2 = result_df.query("Problem=='"+prob+"' and Policy=='2'")
            if len(tab2) == 0:
                continue
            tab2_time = tab2['SolverTotalTime'].values[0] + tab2['SavileRowTotalTime'].values[0]
            baselineTimeOut = baseline["SolverTimeOut"].fillna(1).values+baseline["SavileRowTimeOut"].fillna(1).values
            tab2TimeOut = tab2["SolverTimeOut"].fillna(1).values+tab2["SavileRowTimeOut"].fillna(1).values
            if baselineTimeOut>0 and tab2TimeOut>0:
                if model_classifier:
                    targets[prob] = {'Target': 0, 'Score': 0.0, 'Tab_t': 3600, 'Base_t': 3600}
                else:
                    continue  # drop row
            if baselineTimeOut>0 and tab2TimeOut==0:
                clipped = np.clip(tab2_time, 0, 3600)
                targets[prob] = {'Target': 1, 'Score': 3600-clipped, 'Tab_t': clipped, 'Base_t': 3600}
            elif baselineTimeOut==0 and tab2TimeOut>0:
                clipped = np.clip(baseline_time, 0, 3600)
                targets[prob] = {'Target': 0, 'Score': clipped-3600, 'Tab_t': 3600, 'Base_t': clipped}
            elif baselineTimeOut==0 and tab2TimeOut==0 and np.abs(tab2_time-baseline_time)<1:
                if model_classifier:
                    label = 1 if tab2_time < baseline_time else 0
                    clipped_1 = np.clip(tab2_time, 0, 3600)
                    clipped_2 = np.clip(baseline_time, 0, 3600)
                    targets[prob] = {'Target': label, 'Score': clipped_2-clipped_1, 'Tab_t': clipped_1, 'Base_t': clipped_2}
                else:
                    continue # Skip small differences
            elif baselineTimeOut==0 and tab2TimeOut==0:
                label = 1 if tab2_time < baseline_time else 0
                clipped_1 = np.clip(tab2_time, 0, 3600)
                clipped_2 = np.clip(baseline_time, 0, 3600)
                targets[prob] = {'Target': label, 'Score': clipped_2-clipped_1, 'Tab_t': clipped_1, 'Base_t': clipped_2}
        target_df = pd.DataFrame.from_dict(targets, orient ='index')
        target_df['Problem'] = target_df.index
        target_df['similar'] = target_df.apply(lambda row: 1 if (row['Tab_t']<=60 and row['Base_t']<=60) or np.abs(row['Tab_t'] - row['Base_t']) < 0.1*min([row['Base_t'], row['Tab_t']]) else 0, axis=1)
        df_solvs[tmp_set] = target_df.copy()
        name = tmp_dir_logs.split("/")[-1].split("_")[0]
        dfs = dict()
        if (name == "instance" or name == "problem") and not gnn_flag:
            problems = []
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    test_problems, train_problems, all_test, all_train = pickle.load(pkl_f)
                problems.append([test_problems, train_problems])
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_train)])
            for split in dfs.keys():
                dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
        elif name == "single" and not gnn_flag:
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                complete_df = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test, all_train = pickle.load(pkl_f)
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_train)])
                dfs[split].reset_index().drop(["index"], axis=1)
        elif name == "leave" and not gnn_flag:
            complete_df = pd.DataFrame()
            for file in tmp_files:
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                complete_df = pd.concat([complete_df, pd.DataFrame(all_test)])
            df = complete_df.reset_index().drop(["index"], axis=1)
            dfs['0.1'] = df
        elif name == "instance" and gnn_flag:
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
                all_test["prediction"] = [0 for x in range(len(all_test["Problem"]))]
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
            for split in dfs.keys():
                dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
        elif name == "leave" and gnn_flag:
            complete_df = pd.DataFrame()
            for file in tmp_files:
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
                if len(all_test["prediction"][0][0])==1:
                    all_test["prediction"] = [[x[0] for x in all_test["prediction"][i]] for i in range(len(all_test["prediction"]))]
                complete_df = pd.concat([complete_df, pd.DataFrame(all_test)])
            df = complete_df.reset_index().drop(["index"], axis=1)
            dfs['0.1'] = df
        elif name == "problem" and gnn_flag:
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
                all_test["prediction"] = [0 for x in range(len(all_test["Problem"]))]
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
            for split in dfs.keys():
                dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
        elif name == "single" and gnn_flag:
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                complete_df = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
                if len(all_test["prediction"][0][0])==1:
                    all_test["prediction"] = [[x[0] for x in all_test["prediction"][i]] for i in range(len(all_test["prediction"]))]
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
                dfs[split].reset_index().drop(["index"], axis=1)
        if name == "instance" or name == "problem":
            for df_key in dfs.keys():
                complete_df = dfs[df_key]
                df = pd.DataFrame()
                num_cols = [x for x in complete_df.columns if x not in ["Problem", "train_test", "prediction"]]
                for prob in complete_df['Problem'].unique():
                    prob_df = complete_df.query("Problem=='"+prob+"'")
                    for train_test in prob_df['train_test'].unique():
                        a = prob_df.query("train_test=='"+train_test+"'")
                        tmp_df = a[num_cols].mean().to_frame().T
                        tmp_df["Problem"] = prob
                        tmp_df["train_test"] = train_test
                        tmp_df["prediction"] = 1 if (a["prediction"]==a["Problem"]).sum()>=len(a["Problem"])/2 else 0
                        df = pd.concat([df, tmp_df])
                dfs[df_key] = df.reset_index().drop(["index"], axis=1)
                dfs[df_key]["saved_class_v_heur"] = dfs[df_key]["saved_class"]-dfs[df_key]["saved_heur"]
            if gnn_flag and "instance" in dfs.keys():
                dfs['0.3'] = dfs["instance"]
            elif gnn_flag and "problem" in dfs.keys():
                dfs['0.3'] = dfs["problem"]
        elif name == "single" or name=="leave":
            for df_key in dfs.keys():
                dfs[df_key] = dfs[df_key].groupby(["Problem", "train_test"]).agg({
                    "saved_v_best": 'mean',
                    "saved_class": 'mean',
                    "saved_heur": 'mean',
                    "tot_best": 'mean',
                    "tot_class": 'mean',
                    "tot_heur": 'mean',
                    "tot_no_tab": 'mean'
                })
                dfs[df_key] = dfs[df_key].reset_index()
                dfs[df_key]["saved_class_v_heur"] = dfs[df_key]["saved_class"]-dfs[df_key]["saved_heur"]
        if type_folder == "/single_problem_custom":
            keys_dict = {"minion": "0.1", "kissat": "0.4", "kissat_mdd": "0.1", "chuffed": "0.1"}  # problem class
        if type_folder == "/leave_one_out_custom":
            keys_dict = {"minion": "0.1", "kissat": "0.1", "kissat_mdd": "0.1", "chuffed": "0.1"}  # leave_one_out
        if type_folder == "/instance_custom":
            keys_dict = {"minion": "0.3", "kissat": "0.5", "kissat_mdd": "0.2", "chuffed": "0.3"}  # instance
        df_solvs_class[tmp_set] = dfs[keys_dict[tmp_solv]].copy()
    return df_solvs, df_solvs_class

In [ ]:
# Plots of different settings and subsets of features for a given solver
tmp_solv = "kissat"
dfs_boxs = pd.DataFrame()
for setting in ["single_problem", "leave_one_out"]:
    type_folder = "/"+setting+"_custom"
    _, df_solvs_class = run_setting(tmp_solv, type_folder)
    
    df_list = []
    for subset_t in ["rf_all", "dt_subset", "rf_subset", "dt_all"]:
        df_solvs_class[subset_t]["Problem"] = df_solvs_class[subset_t].apply(lambda x: x["Problem"].split("_")[0], axis=1)
        df_solvs_class[subset_t] = df_solvs_class[subset_t].groupby(["Problem", "train_test"]).agg("mean", numeric_only=True).reset_index()
        
        if setting == "single_problem":
                df_solvs_class[subset_t]["saved_s_best"] = 0
                for prob in df_solvs_class[subset_t].Problem.unique():
                    a = df_solvs_class[subset_t].query("train_test=='train' and Problem=='"+prob+"'")
                    mask = df_solvs_class[subset_t]["Problem"].str.match(prob) & df_solvs_class[subset_t]["train_test"].str.match("train")
                    df_solvs_class[subset_t].loc[mask, "saved_s_best"] = 0 if a["saved_heur"].iloc[0]<=0 else a["saved_heur"]
                    b = df_solvs_class[subset_t].query("train_test=='test' and Problem=='"+prob+"'")
                    mask = df_solvs_class[subset_t]["Problem"].str.match(prob) & df_solvs_class[subset_t]["train_test"].str.match("test")
                    df_solvs_class[subset_t].loc[mask, "saved_s_best"] = 0 if a["saved_heur"].iloc[0]<=0 else b["saved_heur"]
                    df_solvs_class[subset_t].loc[mask, "saved_s_best"] = b["saved_class"] - df_solvs_class[subset_t].loc[mask, "saved_s_best"]
        elif setting == "leave_one_out":
            df_solvs_class[subset_t]["saved_s_best"] = 0
            for prob in df_solvs_class[subset_t].Problem.unique():
                a = df_solvs_class[subset_t].query("train_test=='test' and Problem!='"+prob+"'")
                b = df_solvs_class[subset_t].query("train_test=='test' and Problem=='"+prob+"'")
                mask = df_solvs_class[subset_t]["Problem"].str.match(prob) & df_solvs_class[subset_t]["train_test"].str.match("test")
                df_solvs_class[subset_t].loc[mask, "saved_s_best"] = 0 if a["saved_heur"].sum()<=0 else b["saved_heur"]
                df_solvs_class[subset_t].loc[mask, "saved_s_best"] = b["saved_class"] - df_solvs_class[subset_t].loc[mask, "saved_s_best"]
        elif setting == "instance":
            p_list = set([prob[0] for prob in df_solvs_class[subset_t]["Problem"].str.split("_")])
            prob_sum = dict()
            train_sum = dict()
            for prob in p_list:
                prob_sum[prob] = df_solvs_class[subset_t][df_solvs_class[subset_t]["Problem"].str.contains(prob+"_")].query("train_test=='test'").sum()
                train_sum[prob] = df_solvs_class[subset_t][df_solvs_class[subset_t]["Problem"].str.contains(prob+"_")].query("train_test=='train'").sum()
            df_solvs_class[subset_t] = pd.DataFrame(prob_sum).transpose().drop(["Problem", "prediction"], axis=1)
            train_sum = pd.DataFrame(train_sum).transpose().drop(["Problem", "prediction"], axis=1)
            df_solvs_class[subset_t] = df_solvs_class[subset_t].reset_index()
            df_solvs_class[subset_t]["Problem"] = df_solvs_class[subset_t]["index"]
            df_solvs_class[subset_t]["saved_s_best"] = prob_sum[prob]["saved_class"] if train_sum["saved_heur"].sum()<=0 else prob_sum[prob]["saved_class"]-prob_sum[prob]["saved_heur"]
    
        if "prediction" in df_solvs_class[subset_t].columns:
            df_list.append(df_solvs_class[subset_t].query("train_test=='test'").drop(["prediction", "train_test"], axis=1).copy().assign(Trial=subset_t))
        else:
            df_list.append(df_solvs_class[subset_t].query("train_test=='test'").drop(["train_test"], axis=1).copy().assign(Trial=subset_t))

        if setting == "instance":
            df_list[-1] = df_list[-1].drop(["index"], axis=1).copy().assign(Trial=subset_t)
            cols = [x for x in df_list[-1].columns if x != "Trial" and x!= "Problem"]
            df_list[-1][cols] = df_list[-1][cols].astype(float)
            
        df_list[-1]["tot_class_log"] = np.log(df_list[-1]["tot_class"])
    
        df_list[-1]["tot_class - "+subset_t] = df_list[-1]["tot_class"]
        df_list[-1]["tot_class_log - "+subset_t] = df_list[-1]["tot_class_log"]
        df_list[-1]["saved_class - "+subset_t] = df_list[-1]["saved_class"]
        df_list[-1]["saved_class_v_heur - "+subset_t] = df_list[-1]["saved_class_v_heur"]
        df_list[-1]["saved_s_best - "+subset_t] = df_list[-1]["saved_s_best"]
    
        df_list[-1] = df_list[-1][["tot_class - "+subset_t, "tot_class_log - "+subset_t, "saved_class - "+subset_t, "saved_class_v_heur - "+subset_t, "saved_s_best - "+subset_t, "Problem"]]
    
    dfs_boxs_tmp = df_list[0].merge(df_list[-1], on="Problem")
    dfs_boxs_tmp = dfs_boxs_tmp.merge(df_list[1], on="Problem")
    dfs_boxs_tmp = dfs_boxs_tmp.merge(df_list[2], on="Problem")
    dfs_boxs_tmp = dfs_boxs_tmp.drop(["Problem"], axis=1)
    
    dfs_boxs_tmp["Setting"] = " ".join([x[0].upper()+x[1:] for x in setting.split("_")])
    dfs_boxs_tmp["Setting"] = dfs_boxs_tmp["Setting"].apply(lambda x: "By Instance" if x=="Instance" else x)
    dfs_boxs_tmp["Setting"] = dfs_boxs_tmp["Setting"].apply(lambda x: "Per Problem Class" if x=="Single Problem" else x)
    
    dfs_boxs = pd.concat([dfs_boxs, dfs_boxs_tmp])
    
    print("Finished " + setting)

In [ ]:
import seaborn as sns

threshold = 0

a = dfs_boxs[["Setting"]+["tot_class - "+subset_t for subset_t in ["rf_all", "dt_subset", "rf_subset", "dt_all"]]].copy()
a["tot_class - RF subset"] = a["tot_class - rf_subset"]
a["tot_class - DT subset"] = a["tot_class - dt_subset"]
a["tot_class - DT all"] = a["tot_class - dt_all"]
a["tot_class - RF all"] = a["tot_class - rf_all"]
a = a.drop([x for x in a.columns if "RF" not in x and "DT" not in x and x!="Setting"], axis=1)
for c in a.columns:
    if c != "Setting":
        a[c.split(" - ")[1]] = a[c]
        a = a.drop([c], axis=1)
a = a.reset_index()
a = a.drop(["index"], axis=1)

mdf = pd.melt(a, id_vars=['Setting'], var_name=['Method'])
mask = (mdf["value"]>threshold) | (mdf["value"]<-threshold)
mdf = mdf[mask]

if len(mdf) > 1:
    sns.boxplot(x='Setting', y="value", hue="Method", data=mdf, 
                hue_order=sorted(mdf["Method"].unique()), showmeans=True, showfliers = False);

    plt.title(tmp_solv[0].upper()+" ".join(tmp_solv[1:].split("_")));
    plt.ylabel("Total time (s)");
    plt.tight_layout();
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.savefig("total_feat_settings_boxplot_"+tmp_solv+".jpg", bbox_inches = 'tight');
    plt.show()

a = dfs_boxs[["Setting"]+["tot_class_log - "+subset_t for subset_t in ["rf_all", "dt_subset", "rf_subset", "dt_all"]]].copy()
a["tot_class_log - RF subset"] = a["tot_class_log - rf_subset"]
a["tot_class_log - DT subset"] = a["tot_class_log - dt_subset"]
a["tot_class_log - DT all"] = a["tot_class_log - dt_all"]
a["tot_class_log - RF all"] = a["tot_class_log - rf_all"]
a = a.drop([x for x in a.columns if "RF" not in x and "DT" not in x and x!="Setting"], axis=1)
for c in a.columns:
    if c != "Setting":
        a[c.split(" - ")[1]] = a[c]
        a = a.drop([c], axis=1)
a = a.reset_index()
a = a.drop(["index"], axis=1)

mdf = pd.melt(a, id_vars=['Setting'], var_name=['Method'])
mask = (mdf["value"]>threshold) | (mdf["value"]<-threshold)
mdf = mdf[mask]

if len(mdf) > 1:
    sns.boxplot(x='Setting', y="value", hue="Method", data=mdf, hue_order=sorted(mdf["Method"].unique()), showmeans=True);

    plt.title(tmp_solv[0].upper()+" ".join(tmp_solv[1:].split("_")));
    plt.ylabel("Total time in log scale (s)");
    plt.tight_layout();
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.savefig("total_feat_settings_boxplot_log_"+tmp_solv+".jpg", bbox_inches = 'tight');
    plt.show()

a = dfs_boxs[["Setting"]+["saved_class - "+subset_t for subset_t in ["rf_all", "dt_subset", "rf_subset", "dt_all"]]].copy()
a["saved_class - RF subset"] = a["saved_class - rf_subset"]
a["saved_class - DT subset"] = a["saved_class - dt_subset"]
a["saved_class - DT all"] = a["saved_class - dt_all"]
a["saved_class - RF all"] = a["saved_class - rf_all"]
a = a.drop([x for x in a.columns if "RF" not in x and "DT" not in x and x!="Setting"], axis=1)
for c in a.columns:
    if c != "Setting":
        a[c.split(" - ")[1]] = a[c]
        a = a.drop([c], axis=1)
a = a.reset_index()
a = a.drop(["index"], axis=1)

mdf = pd.melt(a, id_vars=['Setting'], var_name=['Method'])
mask = (mdf["value"]>threshold) | (mdf["value"]<-threshold)
mdf = mdf[mask]

if len(mdf) > 1:
    sns.boxplot(x='Setting', y="value", hue="Method", data=mdf, 
                hue_order=sorted(mdf["Method"].unique()), showmeans=True, showfliers = False);

    plt.title(tmp_solv[0].upper()+" ".join(tmp_solv[1:].split("_")));
    plt.ylabel("Saved time with respect to never using tabulation (s)");
    plt.tight_layout();
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.savefig("saved_feat_settings_boxplot_"+tmp_solv+".jpg", bbox_inches = 'tight');
    plt.show()
    
a = dfs_boxs[["Setting"]+["saved_class_v_heur - "+subset_t for subset_t in ["rf_all", "dt_subset", "rf_subset", "dt_all"]]].copy()
a["saved_class_v_heur - RF subset"] = a["saved_class_v_heur - rf_subset"]
a["saved_class_v_heur - DT subset"] = a["saved_class_v_heur - dt_subset"]
a["saved_class_v_heur - DT all"] = a["saved_class_v_heur - dt_all"]
a["saved_class_v_heur - RF all"] = a["saved_class_v_heur - rf_all"]
a = a.drop([x for x in a.columns if "RF" not in x and "DT" not in x and x!="Setting"], axis=1)
for c in a.columns:
    if c != "Setting":
        a[c.split(" - ")[1]] = a[c]
        a = a.drop([c], axis=1)
a = a.reset_index()
a = a.drop(["index"], axis=1)

mdf = pd.melt(a, id_vars=['Setting'], var_name=['Method'])
mask = (mdf["value"]>threshold) | (mdf["value"]<-threshold)
mdf = mdf[mask]

if len(mdf) > 1:
    sns.boxplot(x='Setting', y="value", hue="Method", data=mdf, 
                hue_order=sorted(mdf["Method"].unique()), showmeans=True, showfliers = False);

    plt.title(tmp_solv[0].upper()+" ".join(tmp_solv[1:].split("_")));
    plt.ylabel("Saved time with respect to the heuristics (s)");
    plt.tight_layout();
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.savefig("saved_v_heur_feat_settings_boxplot_"+tmp_solv+".jpg", bbox_inches = 'tight');
    plt.show()
             
a = dfs_boxs[["Setting"]+["saved_s_best - "+subset_t for subset_t in ["rf_all", "dt_subset", "rf_subset", "dt_all"]]].copy()
a["saved_s_best - RF subset"] = a["saved_s_best - rf_subset"]
a["saved_s_best - DT subset"] = a["saved_s_best - dt_subset"]
a["saved_s_best - DT all"] = a["saved_s_best - dt_all"]
a["saved_s_best - RF all"] = a["saved_s_best - rf_all"]
a = a.drop([x for x in a.columns if "RF" not in x and "DT" not in x and x!="Setting"], axis=1)
for c in a.columns:
    if c != "Setting":
        a[c.split(" - ")[1]] = a[c]
        a = a.drop([c], axis=1)
a = a.reset_index()
a = a.drop(["index"], axis=1)

mdf = pd.melt(a, id_vars=['Setting'], var_name=['Method'])
mask = (mdf["value"]>threshold) | (mdf["value"]<-threshold)
mdf = mdf[mask]

if len(mdf) > 1:
    sns.boxplot(x='Setting', y="value", hue="Method", data=mdf, 
                hue_order=sorted(mdf["Method"].unique()), showmeans=True, showfliers = False);

    plt.title(tmp_solv[0].upper()+" ".join(tmp_solv[1:].split("_")));
    plt.ylabel("Saved time with respect to the single best (s)");
    plt.tight_layout();
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.savefig("saved_s_best_feat_settings_boxplot_"+tmp_solv+".jpg", bbox_inches = 'tight');
    plt.show()

In [ ]:
def run_setting(type_folder):
    df_solvs = dict()
    df_solvs_class = dict()
    for tmp_solv in ["minion", "kissat", "kissat_mdd", "chuffed"]:
        tmp_dir_logs = "../ML_logs/rf_all/"+tmp_solv+type_folder
        tmp_files = os.listdir(tmp_dir_logs)
        tmp_ground_truth = "../Dataset/results_on_off_"+tmp_solv+".csv"
        df_solvs[tmp_solv] = pd.read_csv(tmp_ground_truth)
        df_solvs[tmp_solv].drop_duplicates(["Problem", "Policy"], keep="first", inplace=True)
        targets = dict()
        tot_time_without_tab = 0.0
        tot_time_with_tab = 0.0
        for prob in df_solvs[tmp_solv]["Problem"].unique():
            baseline = df_solvs[tmp_solv].query("Problem=='"+prob+"' and Policy=='baseline'")
            if len(baseline) == 0:
                continue
            baseline_time = baseline['SolverTotalTime'].values[0] + baseline['SavileRowTotalTime'].values[0]
            tab2 = result_df.query("Problem=='"+prob+"' and Policy=='2'")
            if len(tab2) == 0:
                continue
            tab2_time = tab2['SolverTotalTime'].values[0] + tab2['SavileRowTotalTime'].values[0]
            baselineTimeOut = baseline["SolverTimeOut"].fillna(1).values+baseline["SavileRowTimeOut"].fillna(1).values
            tab2TimeOut = tab2["SolverTimeOut"].fillna(1).values+tab2["SavileRowTimeOut"].fillna(1).values
            if baselineTimeOut>0 and tab2TimeOut>0:
                if model_classifier:
                    targets[prob] = {'Target': 0, 'Score': 0.0, 'Tab_t': 3600, 'Base_t': 3600}
                else:
                    continue  # drop row
            if baselineTimeOut>0 and tab2TimeOut==0:
                clipped = np.clip(tab2_time, 0, 3600)
                targets[prob] = {'Target': 1, 'Score': 3600-clipped, 'Tab_t': clipped, 'Base_t': 3600}
            elif baselineTimeOut==0 and tab2TimeOut>0:
                clipped = np.clip(baseline_time, 0, 3600)
                targets[prob] = {'Target': 0, 'Score': clipped-3600, 'Tab_t': 3600, 'Base_t': clipped}
            elif baselineTimeOut==0 and tab2TimeOut==0 and np.abs(tab2_time-baseline_time)<1:
                if model_classifier:
                    label = 1 if tab2_time < baseline_time else 0
                    clipped_1 = np.clip(tab2_time, 0, 3600)
                    clipped_2 = np.clip(baseline_time, 0, 3600)
                    targets[prob] = {'Target': label, 'Score': clipped_2-clipped_1, 'Tab_t': clipped_1, 'Base_t': clipped_2}
                else:
                    continue # Skip small differences
            elif baselineTimeOut==0 and tab2TimeOut==0:# and np.abs(tab2_time-baseline_time)>=1:
                label = 1 if tab2_time < baseline_time else 0
                clipped_1 = np.clip(tab2_time, 0, 3600)
                clipped_2 = np.clip(baseline_time, 0, 3600)
                targets[prob] = {'Target': label, 'Score': clipped_2-clipped_1, 'Tab_t': clipped_1, 'Base_t': clipped_2}
        target_df = pd.DataFrame.from_dict(targets, orient ='index')
        target_df['Problem'] = target_df.index
        target_df['similar'] = target_df.apply(lambda row: 1 if (row['Tab_t']<=60 and row['Base_t']<=60) or np.abs(row['Tab_t'] - row['Base_t']) < 0.1*min([row['Base_t'], row['Tab_t']]) else 0, axis=1)
        df_solvs[tmp_solv] = target_df.copy()
        name = tmp_dir_logs.split("/")[-1].split("_")[0]
        dfs = dict()
        if (name == "instance" or name == "problem") and not gnn_flag:
            problems = []
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    test_problems, train_problems, all_test, all_train = pickle.load(pkl_f)
                problems.append([test_problems, train_problems])
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_train)])
            for split in dfs.keys():
                dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
        elif name == "single" and not gnn_flag:
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                complete_df = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test, all_train = pickle.load(pkl_f)
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_train)])
                dfs[split].reset_index().drop(["index"], axis=1)
        elif name == "leave" and not gnn_flag:
            complete_df = pd.DataFrame()
            for file in tmp_files:
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                complete_df = pd.concat([complete_df, pd.DataFrame(all_test)])
                #complete_df = pd.concat([complete_df, pd.DataFrame(all_train)])
            df = complete_df.reset_index().drop(["index"], axis=1)
            dfs['0.1'] = df
        elif name == "instance" and gnn_flag:
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
                all_test["prediction"] = [0 for x in range(len(all_test["Problem"]))]
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
            for split in dfs.keys():
                dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
        elif name == "leave" and gnn_flag:
            complete_df = pd.DataFrame()
            for file in tmp_files:
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
                if len(all_test["prediction"][0][0])==1:
                    all_test["prediction"] = [[x[0] for x in all_test["prediction"][i]] for i in range(len(all_test["prediction"]))]
                complete_df = pd.concat([complete_df, pd.DataFrame(all_test)])
            df = complete_df.reset_index().drop(["index"], axis=1)
            dfs['0.1'] = df
        elif name == "problem" and gnn_flag:
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
                all_test["prediction"] = [0 for x in range(len(all_test["Problem"]))]
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
            for split in dfs.keys():
                dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
        elif name == "single" and gnn_flag:
            for file in tmp_files:
                split = str(file.split("_")[-1].split(".pkl")[0])
                if split not in dfs.keys():
                    dfs[split] = pd.DataFrame()
                complete_df = pd.DataFrame()
                filename = os.path.join(tmp_dir_logs, file)
                with open(filename, "rb") as pkl_f:
                    all_test = pickle.load(pkl_f)[0]
                all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
                if len(all_test["prediction"][0][0])==1:
                    all_test["prediction"] = [[x[0] for x in all_test["prediction"][i]] for i in range(len(all_test["prediction"]))]
                dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
                dfs[split].reset_index().drop(["index"], axis=1)
        if name == "instance" or name == "problem":
            for df_key in dfs.keys():
                complete_df = dfs[df_key]
                df = pd.DataFrame()
                num_cols = [x for x in complete_df.columns if x not in ["Problem", "train_test", "prediction"]]
                for prob in complete_df['Problem'].unique():
                    prob_df = complete_df.query("Problem=='"+prob+"'")
                    for train_test in prob_df['train_test'].unique():
                        a = prob_df.query("train_test=='"+train_test+"'")
                        tmp_df = a[num_cols].mean().to_frame().T
                        tmp_df["Problem"] = prob
                        tmp_df["train_test"] = train_test
                        tmp_df["prediction"] = 1 if (a["prediction"]==a["Problem"]).sum()>=len(a["Problem"])/2 else 0
                        df = pd.concat([df, tmp_df])
                dfs[df_key] = df.reset_index().drop(["index"], axis=1)
                dfs[df_key]["saved_class_v_heur"] = dfs[df_key]["saved_class"]-dfs[df_key]["saved_heur"]
            if gnn_flag and "instance" in dfs.keys():
                dfs['0.3'] = dfs["instance"]
            elif gnn_flag and "problem" in dfs.keys():
                dfs['0.3'] = dfs["problem"]
        elif name == "single" or name=="leave":
            for df_key in dfs.keys():
                dfs[df_key] = dfs[df_key].groupby(["Problem", "train_test"]).agg({
                    "saved_v_best": 'mean',
                    "saved_class": 'mean',
                    "saved_heur": 'mean',
                    "tot_best": 'mean',
                    "tot_class": 'mean',
                    "tot_heur": 'mean',
                    "tot_no_tab": 'mean'
                })
                dfs[df_key] = dfs[df_key].reset_index()
                dfs[df_key]["saved_class_v_heur"] = dfs[df_key]["saved_class"]-dfs[df_key]["saved_heur"]
        if type_folder == "/single_problem_custom" and not gnn_flag:
            keys_dict = {"minion": "0.1", "kissat": "0.4", "kissat_mdd": "0.1", "chuffed": "0.1"}  # problem class
        elif type_folder == "/leave_one_out_custom" and not gnn_flag:
            keys_dict = {"minion": "0.1", "kissat": "0.1", "kissat_mdd": "0.1", "chuffed": "0.1"}  # leave_one_out
        elif type_folder == "/instance_custom" and not gnn_flag:
            keys_dict = {"minion": "0.3", "kissat": "0.5", "kissat_mdd": "0.2", "chuffed": "0.3"}  # instance
        if not gnn_flag:
            df_solvs_class[tmp_solv] = dfs[keys_dict[tmp_solv]].copy()
        else:
            df_solvs_class[tmp_solv] = dfs[list(dfs.keys())[0]].copy()
    return df_solvs, df_solvs_class

In [ ]:
# Plots of different settings and solvers, against a given setting (no tabulation, heuristics, single best)
dfs_boxs = pd.DataFrame()
for setting in ["single_problem", "leave_one_out"]:
    type_folder = "/"+setting+"_custom"
    _, df_solvs_class = run_setting(type_folder)
    
    df_list = []
    for tmp_solv in ["minion", "kissat", "kissat_mdd", "chuffed"]:
        
        if setting == "single_problem":
                df_solvs_class[tmp_solv]["saved_s_best"] = 0
                for prob in df_solvs_class[tmp_solv].Problem.unique():
                    a = df_solvs_class[tmp_solv].query("train_test=='train' and Problem=='"+prob+"'")
                    mask = df_solvs_class[tmp_solv]["Problem"].str.match(prob) & df_solvs_class[tmp_solv]["train_test"].str.match("train")
                    df_solvs_class[tmp_solv].loc[mask, "saved_s_best"] = 0 if a["saved_heur"].iloc[0]<=0 else a["saved_heur"]
                    b = df_solvs_class[tmp_solv].query("train_test=='test' and Problem=='"+prob+"'")
                    mask = df_solvs_class[tmp_solv]["Problem"].str.match(prob) & df_solvs_class[tmp_solv]["train_test"].str.match("test")
                    df_solvs_class[tmp_solv].loc[mask, "saved_s_best"] = 0 if a["saved_heur"].iloc[0]<=0 else b["saved_heur"]
                    df_solvs_class[tmp_solv].loc[mask, "saved_s_best"] = b["saved_class"] - df_solvs_class[tmp_solv].loc[mask, "saved_s_best"]
        elif setting == "leave_one_out":
            df_solvs_class[tmp_solv]["saved_s_best"] = 0
            for prob in df_solvs_class[tmp_solv].Problem.unique():
                a = df_solvs_class[tmp_solv].query("train_test=='test' and Problem!='"+prob+"'")
                b = df_solvs_class[tmp_solv].query("train_test=='test' and Problem=='"+prob+"'")
                mask = df_solvs_class[tmp_solv]["Problem"].str.match(prob) & df_solvs_class[tmp_solv]["train_test"].str.match("test")
                df_solvs_class[tmp_solv].loc[mask, "saved_s_best"] = 0 if a["saved_heur"].sum()<=0 else b["saved_heur"]
                df_solvs_class[tmp_solv].loc[mask, "saved_s_best"] = b["saved_class"] - df_solvs_class[tmp_solv].loc[mask, "saved_s_best"]
        elif setting == "instance":
            p_list = set([prob[0] for prob in df_solvs_class[tmp_solv]["Problem"].str.split("_")])
            prob_sum = dict()
            for prob in p_list:
                prob_sum[prob] = df_solvs_class[tmp_solv][df_solvs_class[tmp_solv]["Problem"].str.contains(prob+"_")].query("train_test=='test'").sum()
            df_solvs_class[tmp_solv] = pd.DataFrame(prob_sum).transpose().drop(["Problem", "prediction"], axis=1)
            df_solvs_class[tmp_solv] = df_solvs_class[tmp_solv].reset_index()
            df_solvs_class[tmp_solv]["Problem"] = df_solvs_class[tmp_solv]["index"]
            df_solvs_class[tmp_solv]["saved_s_best"] = 0
    
        if "prediction" in df_solvs_class[tmp_solv].columns:
            df_list.append(df_solvs_class[tmp_solv].query("train_test=='test'").drop(["prediction", "train_test"], axis=1).copy().assign(Trial=tmp_solv))
        else:
            df_list.append(df_solvs_class[tmp_solv].query("train_test=='test'").drop(["train_test"], axis=1).copy().assign(Trial=tmp_solv))

        if setting == "instance":
            df_list[-1] = df_list[-1].drop(["index"], axis=1).copy().assign(Trial=tmp_solv)
            cols = [x for x in df_list[-1].columns if x != "Trial" and x!= "Problem"]
            df_list[-1][cols] = df_list[-1][cols].astype(float)
    
        tmp_solv_name = tmp_solv[0].upper()+tmp_solv[1:]
        df_list[-1]["saved_class - "+tmp_solv_name] = df_list[-1]["saved_class"]
        df_list[-1]["saved_class_v_heur - "+tmp_solv_name] = df_list[-1]["saved_class_v_heur"]
        df_list[-1]["saved_s_best - "+tmp_solv_name] = df_list[-1]["saved_s_best"]
    
        df_list[-1] = df_list[-1][["saved_class - "+tmp_solv_name, "saved_class_v_heur - "+tmp_solv_name, 
                                   "saved_s_best - "+tmp_solv_name, "Problem"]]
    
    dfs_boxs_tmp = df_list[0].merge(df_list[1], on="Problem")
    dfs_boxs_tmp = dfs_boxs_tmp.merge(df_list[2], on="Problem")
    dfs_boxs_tmp = dfs_boxs_tmp.merge(df_list[3], on="Problem")
    dfs_boxs_tmp = dfs_boxs_tmp.drop(["Problem"], axis=1)
    
    dfs_boxs_tmp["Setting"] = " ".join([x[0].upper()+x[1:] for x in setting.split("_")])
    dfs_boxs_tmp["Setting"] = dfs_boxs_tmp["Setting"].apply(lambda x: "By Instance" if x=="Instance" else x)
    dfs_boxs_tmp["Setting"] = dfs_boxs_tmp["Setting"].apply(lambda x: "Per Problem Class" if x=="Single Problem" else x)
    
    dfs_boxs = pd.concat([dfs_boxs, dfs_boxs_tmp])
    
    print("Finished " + setting)

In [ ]:
import seaborn as sns
a = dfs_boxs[["Setting"]+["saved_class - "+tmp_solv for tmp_solv in ["Minion", "Kissat", "Kissat_mdd", "Chuffed"]]].copy()
for c in a.columns:
    if c != "Setting":
        a[c.split(" - ")[1]] = a[c]
        a = a.drop([c], axis=1)
a = a.reset_index()
a = a.drop(["index"], axis=1)

mdf = pd.melt(a, id_vars=['Setting'], var_name=['Method'])

if len(mdf) > 1:
    sns.boxplot(x='Setting', y="value", hue="Method", data=mdf, 
                hue_order=sorted(mdf["Method"].unique()), showmeans=True, showfliers = True);

    plt.title("Saved time with respect to never using tabulation");
    plt.ylabel("Saved time (s)");
    plt.tight_layout();
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.savefig("saved_settings_boxplot_"+tmp_solv+".jpg", bbox_inches = 'tight');
    plt.show()
    
a = dfs_boxs[["Setting"]+["saved_class_v_heur - "+tmp_solv for tmp_solv in ["Minion", "Kissat", "Kissat_mdd", "Chuffed"]]].copy()
for c in a.columns:
    if c != "Setting":
        a[c.split(" - ")[1]] = a[c]
        a = a.drop([c], axis=1)
a = a.reset_index()
a = a.drop(["index"], axis=1)

mdf = pd.melt(a, id_vars=['Setting'], var_name=['Method'])

if len(mdf) > 1:
    sns.boxplot(x='Setting', y="value", hue="Method", data=mdf, 
                hue_order=sorted(mdf["Method"].unique()), showmeans=True, showfliers = True);

    plt.title("Saved time with respect to the heuristics");
    plt.ylabel("Saved time (s)");
    plt.tight_layout();
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.savefig("saved_v_heur_settings_boxplot_"+tmp_solv+".jpg", bbox_inches = 'tight');
    plt.show()
             
a = dfs_boxs[["Setting"]+["saved_s_best - "+tmp_solv for tmp_solv in ["Minion", "Kissat", "Kissat_mdd", "Chuffed"]]].copy()
for c in a.columns:
    if c != "Setting":
        a[c.split(" - ")[1]] = a[c]
        a = a.drop([c], axis=1)
a = a.reset_index()
a = a.drop(["index"], axis=1)

mdf = pd.melt(a, id_vars=['Setting'], var_name=['Method'])

if len(mdf) > 1:
    sns.boxplot(x='Setting', y="value", hue="Method", data=mdf, 
                hue_order=sorted(mdf["Method"].unique()), showmeans=True, showfliers = True);

    plt.title("Saved time with respect to the single best");
    plt.ylabel("Saved time (s)");
    plt.tight_layout();
    plt.legend(bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0)
    plt.savefig("saved_s_best_settings_boxplot_"+tmp_solv+".jpg", bbox_inches = 'tight');
    plt.show()

In [ ]:
df_solvs = dict()
df_solvs_class = dict()
for tmp_solv in ["minion", "kissat", "kissat_mdd", "chuffed"]:
    tmp_dir_logs = "../ML_logs/rf_all/"+tmp_solv+type_folder
    tmp_files = os.listdir(tmp_dir_logs)
    tmp_ground_truth = "../Dataset/results_on_off_"+tmp_solv+".csv"
    df_solvs[tmp_solv] = pd.read_csv(tmp_ground_truth)
    df_solvs[tmp_solv].drop_duplicates(["Problem", "Policy"], keep="first", inplace=True)
    targets = dict()
    tot_time_without_tab = 0.0
    tot_time_with_tab = 0.0
    for prob in df_solvs[tmp_solv]["Problem"].unique():
        baseline = df_solvs[tmp_solv].query("Problem=='"+prob+"' and Policy=='baseline'")
        if len(baseline) == 0:
            continue
        baseline_time = baseline['SolverTotalTime'].values[0] + baseline['SavileRowTotalTime'].values[0]
        tab2 = result_df.query("Problem=='"+prob+"' and Policy=='2'")
        if len(tab2) == 0:
            continue
        tab2_time = tab2['SolverTotalTime'].values[0] + tab2['SavileRowTotalTime'].values[0]
        baselineTimeOut = baseline["SolverTimeOut"].fillna(1).values+baseline["SavileRowTimeOut"].fillna(1).values
        tab2TimeOut = tab2["SolverTimeOut"].fillna(1).values+tab2["SavileRowTimeOut"].fillna(1).values
        if baselineTimeOut>0 and tab2TimeOut>0:
            if model_classifier:
                targets[prob] = {'Target': 0, 'Score': 0.0, 'Tab_t': 3600, 'Base_t': 3600}
            else:
                continue  # drop row
        if baselineTimeOut>0 and tab2TimeOut==0:
            clipped = np.clip(tab2_time, 0, 3600)
            targets[prob] = {'Target': 1, 'Score': 3600-clipped, 'Tab_t': clipped, 'Base_t': 3600}
        elif baselineTimeOut==0 and tab2TimeOut>0:
            clipped = np.clip(baseline_time, 0, 3600)
            targets[prob] = {'Target': 0, 'Score': clipped-3600, 'Tab_t': 2*3600, 'Base_t': clipped}
        elif baselineTimeOut==0 and tab2TimeOut==0 and np.abs(tab2_time-baseline_time)<1:
            if model_classifier:
                label = 1 if tab2_time < baseline_time else 0
                clipped_1 = np.clip(tab2_time, 0, 3600)
                clipped_2 = np.clip(baseline_time, 0, 3600)
                targets[prob] = {'Target': label, 'Score': clipped_2-clipped_1, 'Tab_t': clipped_1, 'Base_t': clipped_2}
            else:
                continue # Skip small differences
        elif baselineTimeOut==0 and tab2TimeOut==0:# and np.abs(tab2_time-baseline_time)>=1:
            label = 1 if tab2_time < baseline_time else 0
            clipped_1 = np.clip(tab2_time, 0, 3600)
            clipped_2 = np.clip(baseline_time, 0, 3600)
            targets[prob] = {'Target': label, 'Score': clipped_2-clipped_1, 'Tab_t': clipped_1, 'Base_t': clipped_2}
    target_df = pd.DataFrame.from_dict(targets, orient ='index')
    target_df['Problem'] = target_df.index
    target_df['similar'] = target_df.apply(lambda row: 1 if (row['Tab_t']<=60 and row['Base_t']<=60) or np.abs(row['Tab_t'] - row['Base_t']) < 0.1*min([row['Base_t'], row['Tab_t']]) else 0, axis=1)
    df_solvs[tmp_solv] = target_df.copy()
    name = tmp_dir_logs.split("/")[-1].split("_")[0]
    dfs = dict()
    if (name == "instance" or name == "problem") and not gnn_flag:
        problems = []
        for file in tmp_files:
            split = str(file.split("_")[-1].split(".pkl")[0])
            if split not in dfs.keys():
                dfs[split] = pd.DataFrame()
            filename = os.path.join(tmp_dir_logs, file)
            with open(filename, "rb") as pkl_f:
                test_problems, train_problems, all_test, all_train = pickle.load(pkl_f)
            problems.append([test_problems, train_problems])
            dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
            dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_train)])
        for split in dfs.keys():
            dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
    elif name == "single" and not gnn_flag:
        for file in tmp_files:
            split = str(file.split("_")[-1].split(".pkl")[0])
            if split not in dfs.keys():
                dfs[split] = pd.DataFrame()
            complete_df = pd.DataFrame()
            filename = os.path.join(tmp_dir_logs, file)
            with open(filename, "rb") as pkl_f:
                all_test, all_train = pickle.load(pkl_f)
            dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
            dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_train)])
            dfs[split].reset_index().drop(["index"], axis=1)
    elif name == "leave" and not gnn_flag:
        complete_df = pd.DataFrame()
        for file in tmp_files:
            filename = os.path.join(tmp_dir_logs, file)
            with open(filename, "rb") as pkl_f:
                all_test = pickle.load(pkl_f)[0]
            complete_df = pd.concat([complete_df, pd.DataFrame(all_test)])
            #complete_df = pd.concat([complete_df, pd.DataFrame(all_train)])
        df = complete_df.reset_index().drop(["index"], axis=1)
        dfs['0.1'] = df
    elif name == "instance" and gnn_flag:
        for file in tmp_files:
            split = str(file.split("_")[-1].split(".pkl")[0])
            if split not in dfs.keys():
                dfs[split] = pd.DataFrame()
            filename = os.path.join(tmp_dir_logs, file)
            with open(filename, "rb") as pkl_f:
                all_test = pickle.load(pkl_f)[0]
            all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
            all_test["prediction"] = [0 for x in range(len(all_test["Problem"]))]
            dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
        for split in dfs.keys():
            dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
    elif name == "leave" and gnn_flag:
        complete_df = pd.DataFrame()
        for file in tmp_files:
            filename = os.path.join(tmp_dir_logs, file)
            with open(filename, "rb") as pkl_f:
                all_test = pickle.load(pkl_f)[0]
            all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
            if len(all_test["prediction"][0][0])==1:
                all_test["prediction"] = [[x[0] for x in all_test["prediction"][i]] for i in range(len(all_test["prediction"]))]
            complete_df = pd.concat([complete_df, pd.DataFrame(all_test)])
        df = complete_df.reset_index().drop(["index"], axis=1)
        dfs['0.1'] = df
    elif name == "problem" and gnn_flag:
        for file in tmp_files:
            split = str(file.split("_")[-1].split(".pkl")[0])
            if split not in dfs.keys():
                dfs[split] = pd.DataFrame()
            filename = os.path.join(tmp_dir_logs, file)
            with open(filename, "rb") as pkl_f:
                all_test = pickle.load(pkl_f)[0]
            all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
            all_test["prediction"] = [0 for x in range(len(all_test["Problem"]))]
            dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
        for split in dfs.keys():
            dfs[split] = dfs[split].reset_index().drop(["index"], axis=1)
    elif name == "single" and gnn_flag:
        for file in tmp_files:
            split = str(file.split("_")[-1].split(".pkl")[0])
            if split not in dfs.keys():
                dfs[split] = pd.DataFrame()
            complete_df = pd.DataFrame()
            filename = os.path.join(tmp_dir_logs, file)
            with open(filename, "rb") as pkl_f:
                all_test = pickle.load(pkl_f)[0]
            all_test["train_test"] = ["test" for x in range(len(all_test["Problem"]))]
            if len(all_test["prediction"][0][0])==1:
                all_test["prediction"] = [[x[0] for x in all_test["prediction"][i]] for i in range(len(all_test["prediction"]))]
            dfs[split] = pd.concat([dfs[split], pd.DataFrame(all_test)])
            dfs[split].reset_index().drop(["index"], axis=1)
    if name == "instance" or name == "problem":
        for df_key in dfs.keys():
            complete_df = dfs[df_key]
            df = pd.DataFrame()
            num_cols = [x for x in complete_df.columns if x not in ["Problem", "train_test", "prediction"]]
            for prob in complete_df['Problem'].unique():
                prob_df = complete_df.query("Problem=='"+prob+"'")
                for train_test in prob_df['train_test'].unique():
                    a = prob_df.query("train_test=='"+train_test+"'")
                    tmp_df = a[num_cols].mean().to_frame().T
                    tmp_df["Problem"] = prob
                    tmp_df["train_test"] = train_test
                    tmp_df["prediction"] = 1 if (a["prediction"]==a["Problem"]).sum()>=len(a["Problem"])/2 else 0
                    df = pd.concat([df, tmp_df])
            dfs[df_key] = df.reset_index().drop(["index"], axis=1)
        if gnn_flag and "instance" in dfs.keys():
            dfs['0.3'] = dfs["instance"]
        elif gnn_flag and "problem" in dfs.keys():
            dfs['0.3'] = dfs["problem"]
    keys_dict = {"minion": "0.3", "kissat": "0.2", "kissat_mdd": "0.5", "chuffed": "0.2"}
    df_solvs_class[tmp_solv] = dfs[keys_dict[tmp_solv]].copy()

In [ ]:
import seaborn as sns

df_solvs["minion"]["Best"] = [x.Tab_t if x.Tab_t<x.Base_t else x.Base_t for x in df_solvs["minion"].itertuples()]
df_solvs["kissat"]["Best"] = [x.Tab_t if x.Tab_t<x.Base_t else x.Base_t for x in df_solvs["kissat"].itertuples()]
df_solvs["kissat_mdd"]["Best"] = [x.Tab_t if x.Tab_t<x.Base_t else x.Base_t for x in df_solvs["kissat_mdd"].itertuples()]
df_solvs["chuffed"]["Best"] = [x.Tab_t if x.Tab_t<x.Base_t else x.Base_t for x in df_solvs["chuffed"].itertuples()]

print("Total time (s) for each solver with virtual best classifier for tabulation/heuristics: ")
print("\t Minion:", np.sum(df_solvs["minion"]["Best"]), "s")
print("\t Kissat:", np.sum(df_solvs["kissat"]["Best"]), "s")
print("\t Kissat mdd:", np.sum(df_solvs["kissat_mdd"]["Best"]), "s")
print("\t Chuffed:", np.sum(df_solvs["chuffed"]["Best"]), "s")

tmp_sum = 0
list_res_notab = dict()
for prob in df_solvs["minion"].Problem.unique():
    if prob in df_solvs["kissat"].Problem.values and prob in df_solvs["kissat_mdd"].Problem.values and prob in df_solvs["chuffed"].Problem.values:
        a = min([df_solvs["minion"].query("Problem=='"+prob+"'")["Base_t"].values[0], 
                 df_solvs["kissat"].query("Problem=='"+prob+"'")["Base_t"].values[0], 
                 df_solvs["kissat_mdd"].query("Problem=='"+prob+"'")["Base_t"].values[0], 
                 df_solvs["chuffed"].query("Problem=='"+prob+"'")["Base_t"].values[0]])
        tmp_sum += a
        list_res_notab[prob] = [a]
df_notab = pd.DataFrame.from_dict(list_res_notab).transpose()
df_notab["Base_t"] = df_notab[0]
df_notab.drop([0], axis=1, inplace=True)
print("Total time (s) for each solver with virtual best classifier for solver and without tabulation:", tmp_sum, "s")

tmp_sum = 0
list_res_heur = dict()
for prob in df_solvs["minion"].Problem.unique():
    if prob in df_solvs["kissat"].Problem.values and prob in df_solvs["kissat_mdd"].Problem.values and prob in df_solvs["chuffed"].Problem.values:
        a = min([df_solvs["minion"].query("Problem=='"+prob+"'")["Tab_t"].values[0], 
                 df_solvs["kissat"].query("Problem=='"+prob+"'")["Tab_t"].values[0], 
                 df_solvs["kissat_mdd"].query("Problem=='"+prob+"'")["Tab_t"].values[0], 
                 df_solvs["chuffed"].query("Problem=='"+prob+"'")["Tab_t"].values[0]])
        tmp_sum += a
        list_res_heur[prob] = [a]
df_heur = pd.DataFrame.from_dict(list_res_heur).transpose()
df_heur["Tab_t"] = df_heur[0]
df_heur.drop([0], axis=1, inplace=True)
print("Total time (s) for each solver with virtual best classifier for solver and with the heuristics:", tmp_sum, "s")

tmp_sum = 0
list_res_class = dict()
for prob in df_solvs["minion"].Problem.unique():
    if prob in df_solvs["kissat"].Problem.values and prob in df_solvs["kissat_mdd"].Problem.values and prob in df_solvs["chuffed"].Problem.values:
        tabs = [df_solvs["minion"].query("Problem=='"+prob+"'")["Tab_t"].values[0], 
               df_solvs["kissat"].query("Problem=='"+prob+"'")["Tab_t"].values[0], 
               df_solvs["kissat_mdd"].query("Problem=='"+prob+"'")["Tab_t"].values[0], 
               df_solvs["chuffed"].query("Problem=='"+prob+"'")["Tab_t"].values[0]]
        bases = [df_solvs["minion"].query("Problem=='"+prob+"'")["Base_t"].values[0], 
               df_solvs["kissat"].query("Problem=='"+prob+"'")["Base_t"].values[0], 
               df_solvs["kissat_mdd"].query("Problem=='"+prob+"'")["Base_t"].values[0], 
               df_solvs["chuffed"].query("Problem=='"+prob+"'")["Base_t"].values[0]]
        a = min([tabs[0] if tabs[0]<bases[0] else bases[0], 
                 tabs[1] if tabs[1]<bases[1] else bases[1], 
                 tabs[2] if tabs[2]<bases[2] else bases[2], 
                 tabs[3] if tabs[3]<bases[3] else bases[3]])
        tmp_sum += a
        list_res_class[prob] = [a]
df_class = pd.DataFrame.from_dict(list_res_class).transpose()
df_class["Best"] = df_class[0]
df_class.drop([0], axis=1, inplace=True)
print("Total time (s) for each solver with virtual best classifier for solver and for the heuristics:", tmp_sum, "s")

df1 = pd.DataFrame([df_solvs["minion"]["Tab_t"], df_solvs["minion"]["Base_t"], df_solvs["minion"]["Best"]]).transpose().assign(Trial="Minion")
df2 = pd.DataFrame([df_solvs["kissat"]["Tab_t"], df_solvs["kissat"]["Base_t"], df_solvs["kissat"]["Best"]]).transpose().assign(Trial="kissat")
df3 = pd.DataFrame([df_solvs["kissat_mdd"]["Tab_t"], df_solvs["kissat_mdd"]["Base_t"], df_solvs["kissat_mdd"]["Best"]]).transpose().assign(Trial="kissat_mdd")
df4 = pd.DataFrame([df_solvs["chuffed"]["Tab_t"], df_solvs["chuffed"]["Base_t"], df_solvs["chuffed"]["Best"]]).transpose().assign(Trial="chuffed")
df5 = pd.DataFrame([df_heur["Tab_t"], df_notab["Base_t"], df_class["Best"]]).transpose().assign(Trial="v_best_class")

cdf = pd.concat([df1, df2, df3, df4, df5])
for col in ["Tab_t", "Base_t", "Best"]:
    cdf[col] = np.log(cdf[col])
mdf = pd.melt(cdf, id_vars=['Trial'], var_name=['Method'])
        
if len(mdf) > 1:
    plt.figure(figsize=(15, 8))
    ax = sns.boxplot(x="Trial", y="value", hue="Method", data=mdf, showmeans=True)
    plt.legend(loc="upper left")

    plt.title("Total time needed to solve all instances with each solver and virtual best classifier.");
    plt.ylabel("Total time in log scale (s).");
    plt.tight_layout();
    plt.savefig("solver_class_boxplot.jpg", bbox_inches = 'tight');
    plt.show();

In [ ]:
target_dict_prob = dict()
problems = list(np.unique([x[0] for x in target_df.Problem.str.split("_")]))
for prob in problems:
    target_dict_prob[prob] = [np.sum(target_df[target_df.Problem.str.contains(prob+"_")]["Tab_t"])]
target_df_problem = pd.DataFrame.from_dict(target_dict_prob).T
target_df_problem["Problem"] = target_df_problem.index
target_df_problem["Tab_t"] = target_df_problem[0]

In [ ]:
plt.plot([1, 10, 25, 50, 100], [x for x in agreement_dict.values()])
plt.title("Agreement time for different numbers of tabulated constraints.")
plt.xlabel("Thresholds (number of constraints)")
plt.ylabel("Agreement time (s)")

In [ ]:
plt.plot([1, 10, 25, 50, 100], [x for x in disagreement_dict.values()])
plt.title("Disagreement time for different numbers of tabulated constraints.")
plt.xlabel("Thresholds (number of constraints)")
plt.ylabel("Disagreement time (s)")

In [ ]:
[x for x in np.array(list(disagreement_dict.values()))+np.array(list(agreement_dict.values()))]

In [ ]:
plt.plot([1, 10, 25, 50, 100], [x for x in np.array(list(disagreement_dict.values()))+np.array(list(agreement_dict.values()))])
plt.title("Total time for different numbers of tabulated constraints.")
plt.xlabel("Thresholds (number of constraints)")
plt.ylabel("Total time (s)")

In [ ]:
metric_analyses = False
if metric_analyses:
    plots = [[], [], [], [], [], []]
    for t in [0.1, 0.2, 0.3, 0.5, 0.75, 1.0]:
        for p in [1, 30, 60]:
            target_df['similar'] = target_df.apply(lambda row: 1 if (row['Tab_t']<=p and row['Base_t']<=p) or np.abs(row['Tab_t'] - row['Base_t']) < t*min([row['Base_t'], row['Tab_t']]) else 0, axis=1)
            probs_stats, target_df = get_metrics()
            plots[0].append(np.abs(probs_stats["fp_time"]).sum())
            plots[1].append(np.abs(probs_stats["fn_time"]).sum())
            plots[2].append(np.abs(probs_stats["no_diff_time"]).sum())
            plots[3].append(probs_stats["precision"].mean())
            plots[4].append(probs_stats["recall"].mean())
            plots[5].append(probs_stats["f1"].mean())

In [ ]:
if metric_analyses:
    for i in range(len(plots)):
        if i>2:
            plt.plot(plots[i])
    plt.title("Precision, recall and f1-score.")
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    plt.legend(["precision", "recall", "f1"])
    plt.show()
    plt.plot(np.log(plots[0]))
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    plt.title("Total time of false positives.")
    plt.ylabel("Log scale, time (s)")
    plt.show()
    plt.plot(np.log(plots[1]))
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    plt.title("Total time of false negatives.")
    plt.ylabel("Log scale, time (s)")
    plt.show()
    plt.plot(np.log(plots[2]))
    ax = plt.gca()
    ax.get_xaxis().set_visible(False)
    plt.title("Total time of no diff.")
    plt.ylabel("Log scale, time (s)")
    plt.show()

In [ ]:
if gnn_flag:
    target_df = target_df.dropna()

In [ ]:
from utils import calculateSpeedups, plotOthersScatter
featured_paper = ["bibd", "carSequencing", "molnars", "opd", "pegSolitaireState", "plotting"]
featured = ['PlottingModelB', 'sokoban']
axis_labels = ["Time without tabulation (s)", "Time with classifier (s)"]
base = True
df_speedups = calculateSpeedups(target_df, base)
plotOthersScatter(target_df, "others.pdf", featured, axis_labels, base, "GNN", df_speedups)

axis_labels = ["Time when always using the heuristics (s)", "Time with classifier (s)"]
base = False
df_speedups = calculateSpeedups(target_df, base)
plotOthersScatter(target_df, "others.pdf", featured, axis_labels, base, "GNN", df_speedups)

In [ ]:
seconds = 60
if not gnn_flag:
    titles = {"saved_class": "Time saved for each problem with random forest classifier", 
             "saved_v_best": "Time saved for each problem with virtual best classifier", 
             "tot_heur": "Total time for each problem when always using the heuristics", 
             "tot_no_tab": "Total time for each problem without tabulation."}
else:
    titles = {"saved_class": "Time saved for each problem with Graph Neural Network", 
             "saved_v_best": "Time saved for each problem with virtual best classifier", 
             "tot_heur": "Total time for each problem when always using the heuristics", 
             "tot_no_tab": "Total time for each problem without tabulation."}
saved_thresh = ", when the time saved is more than "+str(seconds)+" seconds."
total_thresh = ", when the total time is more than "+str(seconds)+" seconds."
for col in ["saved_class", "saved_v_best", "tot_heur", "tot_no_tab"]:
    for ext in [".", "thresh"]:
        if ext == "thresh" and col.split("_")[0] == "saved":
            title = titles[col] + saved_thresh
            thresh = seconds
        elif ext == "thresh" and col.split("_")[0] == "tot":
            title = titles[col] + total_thresh
            thresh = seconds
        elif ext == ".":
            title = titles[col] + "."
            thresh = 0
        a = dfs[best_key].query("train_test == 'test'")[["Problem", col]]
        a = a[a[col]>thresh]
        plt.figure(figsize=(20, 6))
        plt.bar(np.arange(0, len(a["Problem"]), 1), np.log(a[col]))
        plt.title(title)
        plt.xlabel("Problem")
        plt.ylabel("Time (s), logaritmic scale")
        plt.xticks(ticks=np.arange(0, len(a["Problem"]), 1), labels=a["Problem"]);
        plt.savefig(col+"_"+ext+".png", bbox_inches = 'tight')

In [ ]:
figure, axis = plt.subplots(len(dfs[best_key].query("train_test == 'test'")), 2, figsize=(12,200))
cont = 0
for _, row in dfs[best_key].query("train_test == 'test'").iterrows():
    a = row.drop(["Problem", "train_test", "prediction"], axis = 0)
    b = a[['saved_v_best', 'saved_class', 'saved_heur']]
    c = a[['tot_best', 'tot_class', 'tot_heur', 'tot_no_tab']]
    axis[cont, 0].bar([0, 1, 2], b)
    axis[cont, 0].set_title("Total time saved for problem "+row["Problem"])
    axis[cont, 0].set_xticks(ticks=np.arange(0,len(b),1), labels=['saved_v_best', 'saved_class', 'saved_heur'])
    
    axis[cont, 1].bar([0, 1, 2, 3], c)
    axis[cont, 1].set_title("Total time for problem "+row["Problem"])
    axis[cont, 1].set_xticks(ticks=np.arange(0,len(c),1), labels=['tot_best', 'tot_class', 'tot_heur', 'tot_no_tab'])
    cont+=1
plt.show()

In [ ]:
cont = 0
for _, row in dfs[best_key].query("train_test == 'test'").iterrows():
    if row["tot_heur"] >= row["tot_no_tab"]:
        cont+=1
figure, axis = plt.subplots(cont, 1, figsize=(12,80))
cont = 0
all_list = {}
for _, row in dfs[best_key].query("train_test == 'test'").iterrows():
    a = row.drop(["Problem", "train_test", "prediction"], axis = 0)
    if a["tot_heur"] < a["tot_no_tab"]:
        continue
    b = a[['saved_v_best', 'saved_class', 'saved_heur']]
    c = a[['tot_best', 'tot_class', 'tot_heur', 'tot_no_tab']]
    
    axis[cont].bar([0, 1, 2, 3], c)
    axis[cont].set_title("Total time for problem "+row["Problem"])
    axis[cont].set_xticks(ticks=np.arange(0,len(c),1), labels=['tot_best', 'tot_class', 'tot_heur', 'tot_no_tab'])
    cont+=1
    all_list[row["Problem"]] = a["tot_heur"] - a["tot_no_tab"]
print(all_list)

In [ ]:
for df_key in dfs.keys():
    print("Split: ", df_key, "\n", dfs[df_key].drop(["Problem", "train_test"], axis = 1).describe(), "\n")

In [ ]:
if not gnn_flag:
    for df_key in dfs:
        df = dfs[df_key]
        plt.figure(figsize=(8, 8));
        ax = plt.gca()
        if name == "leave" or name == "single":
            max_time = int(max(df.query("train_test=='train'")['tot_best'].max(), df.query("train_test=='train'")['tot_heur'].max()))
        ax.scatter(df.query("train_test=='train'")['tot_heur'], df.query("train_test=='train'")['tot_best'], c="black", marker="x");
        ax.set_yscale('symlog')
        ax.set_xscale('symlog')
        if name == "leave" or name == "single":
            plt.plot([0, max_time+5], [0, max_time+5], c="black");
        else:
            plt.plot([0, 4800+5], [0, 4800+5], c="black");
        if name != 'leave':
            plt.title("Total time for problems in train set with split="+df_key);
            plt.ylabel("Total time of virtual best classifier (s) with split="+df_key);
            plt.xlabel("Total time when always using the heuristics (s) with split="+df_key);
        else:
            plt.title("Total time for problems in train set.");
            plt.ylabel("Total time of virtual best classifier (s).");
            plt.xlabel("Total time when always using the heuristics (s).");
        plt.tight_layout();
        if name == "leave" or name == "single":
            plt.xlim(left=0, right=max_time);
            plt.ylim(bottom=0, top=max_time);
        else:
            plt.xlim(left=0, right=4800);
            plt.ylim(bottom=0, top=4800);
        #plt.savefig("best_heuristics_"+df_key+".jpg", bbox_inches = 'tight')
        plt.show();

In [ ]:
for df_key in [best_key]:  # dfs.keys():
    df = dfs[df_key]
    plt.figure(figsize=(6, 6));
    ax = plt.gca()
    if name == "leave" or name == "single":
        max_time = int(max(df.query("train_test=='test'")['tot_best'].max(), df.query("train_test=='test'")['tot_class'].max()))
    ax.scatter(df.query("train_test=='test'")['tot_best'], df.query("train_test=='test'")['tot_class'], c="black", marker="x");
    ax.set_yscale('symlog')
    ax.set_xscale('symlog')
    if name == "leave" or name == "single":
        plt.plot([0, max_time+5], [0, max_time+5], c="black");
    else:
        plt.plot([0, 4800+5], [0, 4800+5], c="black");
    if name != 'leave':
        plt.title("Total time for problems in test set with split="+df_key);
        plt.ylabel("Total time of random forest classifier (s) with split="+df_key);
        plt.xlabel("Total time of virtual best classifier (s) with split="+df_key);
    else:
        plt.title("Total time for problems in test set.");
        plt.ylabel("Total time of random forest classifier (s).");
        plt.xlabel("Total time of virtual best classifier (s).");
    plt.tight_layout();
    if name == "leave" or name == "single":
        plt.xlim(left=0, right=max_time);
        plt.ylim(bottom=0, top=max_time);
    else:
        plt.xlim(left=0, right=4800);
        plt.ylim(bottom=0, top=4800);
    plt.show();

In [ ]:
if not gnn_flag:
    for df_key in dfs:
        df = dfs[df_key]
        plt.figure(figsize=(8, 8));
        ax = plt.gca()
        if name == "leave" or name == "single":
            max_time = int(max(df.query("train_test=='train'")['tot_best'].max(), df.query("train_test=='train'")['tot_class'].max()))
        ax.scatter(df.query("train_test=='train'")['tot_best'], df.query("train_test=='train'")['tot_class'], c="black", marker="x");
        ax.set_yscale('symlog')
        ax.set_xscale('symlog')
        if name == "leave" or name == "single":
            plt.plot([0, max_time+5], [0, max_time+5], c="black");
        else:
            plt.plot([0, 4800+5], [0, 4800+5], c="black");
        if name != 'leave':
            plt.title("Total time for problems in train set with split="+df_key);
            plt.ylabel("Total time of random forest classifier (s) with split="+df_key);
            plt.xlabel("Total time of virtual best classifier (s) with split="+df_key);
        else:
            plt.title("Total time for problems in train set.");
            plt.ylabel("Total time of random forest classifier (s).");
            plt.xlabel("Total time of virtual best classifier (s).");
        plt.tight_layout();
        if name == "leave" or name == "single":
            plt.xlim(left=0, right=max_time);
            plt.ylim(bottom=0, top=max_time);
        else:
            plt.xlim(left=0, right=4800);
            plt.ylim(bottom=0, top=4800);
        plt.show();

In [ ]:
for df_key in dfs:
    df = dfs[df_key]
    print(df_key,": ",df.query("train_test=='test'")[df.query("train_test=='test'")['tot_heur']+1< df.query("train_test=='test'")['tot_class']].Problem.apply(lambda x: x.split("_")[0]).unique())

In [ ]:
if not gnn_flag:
    for df_key in dfs:
        df = dfs[df_key]
        print(df_key,": ",df.query("train_test=='train'")[df.query("train_test=='train'")['tot_heur']+1< df.query("train_test=='train'")['tot_class']].Problem.apply(lambda x: x.split("_")[0]).unique())